# EigenSpace Mapping: 53-TET Chord Positions

This notebook maps **53-TET (53 equal temperament)** chords to the **EigenSpace** coordinate system.

## EigenSpace Coordinate System
- **(α, β, γ)** = frequency ratios of the 2nd, 3rd, and 4th notes relative to the root
- **Constraint**: α ≤ β ≤ γ (tetrahedron region, eliminates redundant permutations)
- **Range**: 1.0 to 2.0 (one octave above root)

## 53-TET System
- 53 equal divisions of the octave
- Step size = 2^(1/53) ≈ 1.01316 (≈ 22.64 cents)
- Closely approximates just intonation intervals
- Superior to 12-TET for harmonic accuracy

## Goals
1. Define all 53-TET chord positions in EigenSpace
2. Find local minima (harmonic nodes) in the dissonance map  
3. Compare 53-TET chords with 12-TET positions
4. Create reference data for microtonal augmentation pipeline

In [153]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.ndimage import minimum_filter, maximum_filter
import os
from typing import List, Tuple, Dict, Optional
import json

# ============================================================================
# 53-TET FUNDAMENTAL FUNCTIONS
# ============================================================================

def get_53tet_ratio(steps: int) -> float:
    """
    Convert 53-TET steps to frequency ratio.
    
    53-TET divides the octave (2:1) into 53 equal parts.
    Each step = 2^(1/53) ≈ 1.01316 ratio (≈ 22.64 cents)
    
    Args:
        steps: Number of 53-TET steps from root (0-53)
    
    Returns:
        Frequency ratio relative to root
    """
    return 2.0 ** (steps / 53.0)


def get_12tet_ratio(semitones: int) -> float:
    """
    Convert 12-TET semitones to frequency ratio.
    
    Args:
        semitones: Number of semitones from root (0-12)
    
    Returns:
        Frequency ratio relative to root
    """
    return 2.0 ** (semitones / 12.0)


# Helper shorthand for ratio conversion
r = get_53tet_ratio
r12 = get_12tet_ratio

print("53-TET step examples:")
print(f"  Major third (18 steps):  {r(18):.6f}  (vs JI 5/4 = {5/4:.6f})")
print(f"  Perfect fifth (31 steps): {r(31):.6f}  (vs JI 3/2 = {3/2:.6f})")
print(f"  Major seventh (49 steps): {r(49):.6f}  (vs JI 15/8 = {15/8:.6f})")

53-TET step examples:
  Major third (18 steps):  1.265426  (vs JI 5/4 = 1.250000)
  Perfect fifth (31 steps): 1.499941  (vs JI 3/2 = 1.500000)
  Major seventh (49 steps): 1.898064  (vs JI 15/8 = 1.875000)


In [154]:
# ============================================================================
# 53-TET INTERVAL DEFINITIONS
# ============================================================================
# These match exactly the JavaScript definitions from Harmonic_Eigenspace.js

# THIRDS (from root) - 53-TET step values
THIRDS_53 = {
    'sm': 11,   # super-minor (wolf third)
    'vm': 12,   # sub-minor
    'm':  13,   # minor third (300 cents in 12-TET → 13.25 steps)
    '^m': 14,   # super-minor / neutral low
    'n':  15,   # neutral third
    'N':  16,   # neutral third high
    'vM': 17,   # sub-major
    'M':  18,   # major third (400 cents → 17.67 steps, rounds to 18)
    '^M': 19,   # super-major
    'SM': 20,   # wide major (septimal)
}

# FIFTHS (from root) - 53-TET step values  
FIFTHS_53 = {
    'subdim': 29,  # sub-diminished
    'dim':    30,  # diminished fifth
    'vP':     30,  # sub-perfect (same as dim)
    'P':      31,  # perfect fifth (700 cents → 30.92 steps)
    '^P':     32,  # super-perfect
    'aug':    32,  # augmented fifth (same as ^P)
    'upaug':  33,  # up-augmented
}

# SEVENTHS (from root) - 53-TET step values
SEVENTHS_53 = {
    'sm': 42,  # super-diminished 7th
    'vm': 43,  # sub-minor 7th
    'm':  44,  # minor seventh (1000 cents → 44.17 steps)
    '^m': 45,  # super-minor 7th
    'n':  46,  # neutral 7th
    'N':  47,  # neutral 7th high
    'vM': 48,  # sub-major 7th
    'M':  49,  # major seventh (1100 cents → 48.58 steps)
    '^M': 50,  # super-major 7th
    'SM': 51,  # wide major 7th
}

print("53-TET Interval Mappings:")
print(f"\nThirds (steps → ratio → cents):")
for name, steps in THIRDS_53.items():
    ratio = r(steps)
    cents = 1200 * np.log2(ratio)
    print(f"  {name:>3}: {steps:2d} steps → {ratio:.5f} → {cents:.1f}¢")

print(f"\nFifths:")
for name, steps in FIFTHS_53.items():
    ratio = r(steps)
    cents = 1200 * np.log2(ratio)
    print(f"  {name:>6}: {steps:2d} steps → {ratio:.5f} → {cents:.1f}¢")

print(f"\nSevenths:")
for name, steps in SEVENTHS_53.items():
    ratio = r(steps)
    cents = 1200 * np.log2(ratio)
    print(f"  {name:>3}: {steps:2d} steps → {ratio:.5f} → {cents:.1f}¢")

53-TET Interval Mappings:

Thirds (steps → ratio → cents):
   sm: 11 steps → 1.15472 → 249.1¢
   vm: 12 steps → 1.16992 → 271.7¢
    m: 13 steps → 1.18533 → 294.3¢
   ^m: 14 steps → 1.20093 → 317.0¢
    n: 15 steps → 1.21674 → 339.6¢
    N: 16 steps → 1.23276 → 362.3¢
   vM: 17 steps → 1.24898 → 384.9¢
    M: 18 steps → 1.26543 → 407.5¢
   ^M: 19 steps → 1.28208 → 430.2¢
   SM: 20 steps → 1.29896 → 452.8¢

Fifths:
  subdim: 29 steps → 1.46122 → 656.6¢
     dim: 30 steps → 1.48045 → 679.2¢
      vP: 30 steps → 1.48045 → 679.2¢
       P: 31 steps → 1.49994 → 701.9¢
      ^P: 32 steps → 1.51969 → 724.5¢
     aug: 32 steps → 1.51969 → 724.5¢
   upaug: 33 steps → 1.53969 → 747.2¢

Sevenths:
   sm: 42 steps → 1.73202 → 950.9¢
   vm: 43 steps → 1.75482 → 973.6¢
    m: 44 steps → 1.77792 → 996.2¢
   ^m: 45 steps → 1.80132 → 1018.9¢
    n: 46 steps → 1.82504 → 1041.5¢
    N: 47 steps → 1.84906 → 1064.2¢
   vM: 48 steps → 1.87340 → 1086.8¢
    M: 49 steps → 1.89806 → 1109.4¢
   ^M: 50 steps → 1.

In [ ]:
# ============================================================================
# 53-TET CHORD DEFINITIONS - EXACT PORT FROM Harmonic_Eigenspace.js
# ============================================================================

def get_53tet_chord_positions() -> List[Tuple[str, float, float, float]]:
    """
    Get all 53-TET chord positions in EigenSpace (α, β, γ) coordinates.
    
    DIRECT PORT from get53TETChordPositions() in Harmonic_Eigenspace.js
    
    Structure: Each chord is [name, r(third), r(fifth), r(seventh)]
    - Organized by third type (SM=20 down to sm=11)
    - Each third has 10 seventh variants
    - Most use perfect fifth (31), half-dim uses 26
    
    Returns:
        List of (name, α, β, γ) tuples
    """
    chords = []
    
    # ===== SUPER-MAJOR THIRD (SM=20) COMBINATIONS =====
    chords.extend([
        ("SMSM7",   r(20), r(31), r(51)),
        ("SM^M7",   r(20), r(31), r(50)),
        ("SMmaj7",  r(20), r(31), r(49)),
        ("SMvM7",   r(20), r(31), r(48)),
        ("SMN7",    r(20), r(31), r(47)),
        ("SMn7",    r(20), r(31), r(46)),
        ("SM^m7",   r(20), r(31), r(45)),
        ("SMm7",    r(20), r(31), r(44)),
        ("SMvm7",   r(20), r(31), r(43)),
        ("SMsm7",   r(20), r(31), r(42)),
    ])
    
    # ===== UP-MAJOR THIRD (^M=19) COMBINATIONS =====
    chords.extend([
        ("^MSM7",   r(19), r(31), r(51)),
        ("^M^M7",   r(19), r(31), r(50)),
        ("^Mmaj7",  r(19), r(31), r(49)),
        ("^MvM7",   r(19), r(31), r(48)),
        ("^MN7",    r(19), r(31), r(47)),
        ("^Mn7",    r(19), r(31), r(46)),
        ("^M^m7",   r(19), r(31), r(45)),
        ("^Mm7",    r(19), r(31), r(44)),
        ("^Mvm7",   r(19), r(31), r(43)),
        ("^Msm7",   r(19), r(31), r(42)),
    ])
    
    # ===== MAJOR THIRD (M=18) COMBINATIONS =====
    chords.extend([
        ("MSM7",    r(18), r(31), r(51)),
        ("M^M7",    r(18), r(31), r(50)),
        ("maj7",    r(18), r(31), r(49)),
        ("MvM7",    r(18), r(31), r(48)),
        ("MN7",     r(18), r(31), r(47)),
        ("Mn7",     r(18), r(31), r(46)),
        ("M^m7",    r(18), r(31), r(45)),
        ("Mm7",     r(18), r(31), r(44)),
        ("Mvm7",    r(18), r(31), r(43)),
        ("Msm7",    r(18), r(31), r(42)),
    ])
    
    # ===== DOWN-MAJOR THIRD (vM=17) COMBINATIONS =====
    chords.extend([
        ("vMSM7",   r(17), r(31), r(51)),
        ("vM^M7",   r(17), r(31), r(50)),
        ("vMmaj7",  r(17), r(31), r(49)),
        ("vMvM7",   r(17), r(31), r(48)),
        ("vMN7",    r(17), r(31), r(47)),
        ("vMn7",    r(17), r(31), r(46)),
        ("vM^m7",   r(17), r(31), r(45)),
        ("vMm7",    r(17), r(31), r(44)),
        ("vMvm7",   r(17), r(31), r(43)),
        ("vMsm7",   r(17), r(31), r(42)),
    ])
    
    # ===== NEUTRAL-MAJOR THIRD (N=16) COMBINATIONS =====
    chords.extend([
        ("NSM7",    r(16), r(31), r(51)),
        ("N^M7",    r(16), r(31), r(50)),
        ("Nmaj7",   r(16), r(31), r(49)),
        ("NvM7",    r(16), r(31), r(48)),
        ("NN7",     r(16), r(31), r(47)),
        ("Nn7",     r(16), r(31), r(46)),
        ("N^m7",    r(16), r(31), r(45)),
        ("Nm7",     r(16), r(31), r(44)),
        ("Nvm7",    r(16), r(31), r(43)),
        ("Nsm7",    r(16), r(31), r(42)),
    ])
    
    # ===== NEUTRAL-MINOR THIRD (n=15) COMBINATIONS =====
    chords.extend([
        ("nSM7",    r(15), r(31), r(51)),
        ("n^M7",    r(15), r(31), r(50)),
        ("nmaj7",   r(15), r(31), r(49)),
        ("nvM7",    r(15), r(31), r(48)),
        ("nN7",     r(15), r(31), r(47)),
        ("nn7",     r(15), r(31), r(46)),
        ("n^m7",    r(15), r(31), r(45)),
        ("nm7",     r(15), r(31), r(44)),
        ("nvm7",    r(15), r(31), r(43)),
        ("nsm7",    r(15), r(31), r(42)),
    ])
    
    # ===== UP-MINOR THIRD (^m=14) COMBINATIONS =====
    chords.extend([
        ("^mSM7",   r(14), r(31), r(51)),
        ("^m^M7",   r(14), r(31), r(50)),
        ("^mmaj7",  r(14), r(31), r(49)),
        ("^mvM7",   r(14), r(31), r(48)),
        ("^mN7",    r(14), r(31), r(47)),
        ("^mn7",    r(14), r(31), r(46)),
        ("^m^m7",   r(14), r(31), r(45)),
        ("^mm7",    r(14), r(31), r(44)),
        ("^mvm7",   r(14), r(31), r(43)),
        ("^msm7",   r(14), r(31), r(42)),
    ])
    
    # ===== MINOR THIRD (m=13) COMBINATIONS =====
    chords.extend([
        ("mSM7",    r(13), r(31), r(51)),
        ("m^M7",    r(13), r(31), r(50)),
        ("mmaj7",   r(13), r(31), r(49)),
        ("mvM7",    r(13), r(31), r(48)),
        ("mN7",     r(13), r(31), r(47)),
        ("mn7",     r(13), r(31), r(46)),
        ("m^m7",    r(13), r(31), r(45)),
        ("m7",      r(13), r(31), r(44)),
        ("mvm7",    r(13), r(31), r(43)),
        ("msm7",    r(13), r(31), r(42)),
    ])
    
    # ===== DOWN-MINOR THIRD (vm=12) COMBINATIONS =====
    chords.extend([
        ("vmSM7",   r(12), r(31), r(51)),
        ("vm^M7",   r(12), r(31), r(50)),
        ("vmmaj7",  r(12), r(31), r(49)),
        ("vmvM7",   r(12), r(31), r(48)),
        ("vmN7",    r(12), r(31), r(47)),
        ("vmn7",    r(12), r(31), r(46)),
        ("vm^m7",   r(12), r(31), r(45)),
        ("vm7",     r(12), r(31), r(44)),
        ("vmvm7",   r(12), r(31), r(43)),
        ("vmsm7",   r(12), r(31), r(42)),
    ])
    
    # ===== SUB-MINOR THIRD (sm=11) COMBINATIONS =====
    chords.extend([
        ("smSM7",   r(11), r(31), r(51)),
        ("sm^M7",   r(11), r(31), r(50)),
        ("smmaj7",  r(11), r(31), r(49)),
        ("smvM7",   r(11), r(31), r(48)),
        ("smN7",    r(11), r(31), r(47)),
        ("smn7",    r(11), r(31), r(46)),
        ("sm^m7",   r(11), r(31), r(45)),
        ("sm7",     r(11), r(31), r(44)),
        ("smvm7",   r(11), r(31), r(43)),
        ("smsm7",   r(11), r(31), r(42)),
    ])
    
    # ===== HALF-DIMINISHED UP-MINOR (^m=14, fifth=26) =====
    chords.extend([
        ("øSM7",    r(14), r(26), r(51)),
        ("ø^M7",    r(14), r(26), r(50)),
        ("ømaj7",   r(14), r(26), r(49)),
        ("øvM7",    r(14), r(26), r(48)),
        ("øN7",     r(14), r(26), r(47)),
        ("øn7",     r(14), r(26), r(46)),
        ("ø^m7",    r(14), r(26), r(45)),
        ("ø7",      r(14), r(26), r(44)),
        ("øvm7",    r(14), r(26), r(43)),
        ("øsm7",    r(14), r(26), r(42)),
    ])
    
    # ===== HALF-DIMINISHED MINOR (m=13, fifth=26) =====
    chords.extend([
        ("øS7",     r(13), r(26), r(51)),
        ("ø^M7-m",  r(13), r(26), r(50)),
        ("ømaj7-m", r(13), r(26), r(49)),
        ("øvM7-m",  r(13), r(26), r(48)),
        ("øNM7",    r(13), r(26), r(47)),
        ("øn7-m",   r(13), r(26), r(46)),
        ("øv7",     r(13), r(26), r(45)),
        ("ø7-m",    r(13), r(26), r(44)),
        ("øvm7-m",  r(13), r(26), r(43)),
        ("øsm7-m",  r(13), r(26), r(42)),
    ])
    
    # ===== HALF-DIMINISHED DOWN-MINOR (vm=12, fifth=26) =====
    chords.extend([
        ("vøS7",    r(12), r(26), r(51)),
        ("vø^M7",   r(12), r(26), r(50)),
        ("vømaj7",  r(12), r(26), r(49)),
        ("vøvM7",   r(12), r(26), r(48)),
        ("vøN7",    r(12), r(26), r(47)),
        ("vøn7",    r(12), r(26), r(46)),
        ("vø^m7",   r(12), r(26), r(45)),
        ("vø7",     r(12), r(26), r(44)),
        ("vøvm7",   r(12), r(26), r(43)),
        ("vøsm7",   r(12), r(26), r(42)),
    ])
    
    # ===== HALF-DIMINISHED SUB-MINOR (sm=11, fifth=26) =====
    chords.extend([
        ("søS7",    r(11), r(26), r(51)),
        ("sø^M7",   r(11), r(26), r(50)),
        ("sømaj7",  r(11), r(26), r(49)),
        ("søvM7",   r(11), r(26), r(48)),
        ("søN7",    r(11), r(26), r(47)),
        ("søn7",    r(11), r(26), r(46)),
        ("sø^m7",   r(11), r(26), r(45)),
        ("sø7",     r(11), r(26), r(44)),
        ("søvm7",   r(11), r(26), r(43)),
        ("søsm7",   r(11), r(26), r(42)),
    ])
    
    # ===== AUGMENTED (M=18, fifth=35) =====
    chords.extend([
        ("M+S7",    r(18), r(35), r(51)),
        ("M+^M7",   r(18), r(35), r(50)),
        ("M+maj7",  r(18), r(35), r(49)),
        ("M+vM7",   r(18), r(35), r(48)),
        ("M+NM7",   r(18), r(35), r(47)),
        ("M+N7",    r(18), r(35), r(46)),
        ("M+n7",    r(18), r(35), r(45)),
        ("M+m7",    r(18), r(35), r(44)),
        ("M+vm7",   r(18), r(35), r(43)),
        ("M+sm7",   r(18), r(35), r(42)),
    ])
    
    # ===== AUGMENTED SUPER-MAJOR (SM=20, fifth=35) =====
    chords.extend([
        ("SM+S7",   r(20), r(35), r(51)),
        ("SM+^M7",  r(20), r(35), r(50)),
        ("SM+maj7", r(20), r(35), r(49)),
        ("SM+vM7",  r(20), r(35), r(48)),
        ("SM+NM7",  r(20), r(35), r(47)),
        ("SM+N7",   r(20), r(35), r(46)),
        ("SM+n7",   r(20), r(35), r(45)),
        ("SM+m7",   r(20), r(35), r(44)),
        ("SM+vm7",  r(20), r(35), r(43)),
        ("SM+sm7",  r(20), r(35), r(42)),
    ])
    
    # ===== AUGMENTED SUS4 (sus4=22, fifth=35) =====
    chords.extend([
        ("sus4+S7",   r(22), r(35), r(51)),
        ("sus4+^M7",  r(22), r(35), r(50)),
        ("sus4+maj7", r(22), r(35), r(49)),
        ("sus4+vM7",  r(22), r(35), r(48)),
        ("sus4+NM7",  r(22), r(35), r(47)),
        ("sus4+N7",   r(22), r(35), r(46)),
        ("sus4+n7",   r(22), r(35), r(45)),
        ("sus4+m7",   r(22), r(35), r(44)),
        ("sus4+vm7",  r(22), r(35), r(43)),
        ("sus4+sm7",  r(22), r(35), r(42)),
    ])
    
    # ===== SUSPENDED =====
    chords.extend([
        ("7sus4",   r(22), r(31), r(44)),
        ("sus2",    r(9),  r(22), r(31)),
    ])
    
    # ===== TRIADS (γ = β) — the seventh collapses to the fifth =====
    # A triad = root × {1, α, β} — only 3 notes, so γ = β.
    # This places triads on the γ=β diagonal of the EigenSpace tetrahedron.
    
    # Perfect fifth triads (fifth=31)
    for step, name in [(11,"sm"), (12,"vm"), (13,"m"), (14,"^m"), (15,"n"),
                        (16,"N"), (17,"vM"), (18,"M"), (19,"^M"), (20,"SM"), (22,"sus4")]:
        chords.append((f"{name}P", r(step), r(31), r(31)))
    
    # Half-diminished triads (fifth=26)
    for step, name in [(11,"sø"), (12,"vø"), (13,"ø-m"), (14,"ø")]:
        chords.append((f"{name}tri", r(step), r(26), r(26)))
    
    # Augmented triads (fifth=35)
    chords.append(("M+tri", r(18), r(35), r(35)))
    chords.append(("SM+tri", r(20), r(35), r(35)))
    chords.append(("sus4+tri", r(22), r(35), r(35)))
    
    return chords


# Generate the chord list
chords_53tet = get_53tet_chord_positions()
print(f"Total 53-TET chords defined: {len(chords_53tet)}")
print("\nFirst 15 chords (Super-Major third family):")
for name, alpha, beta, gamma in chords_53tet[:15]:
    print(f"  {name:12s}: α={alpha:.5f}, β={beta:.5f}, γ={gamma:.5f}")
print("\n...")
print("\nHalf-diminished family (ø7 variants):")
for name, alpha, beta, gamma in chords_53tet[100:110]:
    print(f"  {name:12s}: α={alpha:.5f}, β={beta:.5f}, γ={gamma:.5f}")

Total 53-TET chords defined: 168

First 15 chords (Super-Major third family):
  SMSM7       : α=1.29896, β=1.49994, γ=1.94837
  SM^M7       : α=1.29896, β=1.49994, γ=1.92305
  SMmaj7      : α=1.29896, β=1.49994, γ=1.89806
  SMvM7       : α=1.29896, β=1.49994, γ=1.87340
  SMN7        : α=1.29896, β=1.49994, γ=1.84906
  SMn7        : α=1.29896, β=1.49994, γ=1.82504
  SM^m7       : α=1.29896, β=1.49994, γ=1.80132
  SMm7        : α=1.29896, β=1.49994, γ=1.77792
  SMvm7       : α=1.29896, β=1.49994, γ=1.75482
  SMsm7       : α=1.29896, β=1.49994, γ=1.73202
  ^MSM7       : α=1.28208, β=1.49994, γ=1.94837
  ^M^M7       : α=1.28208, β=1.49994, γ=1.92305
  ^Mmaj7      : α=1.28208, β=1.49994, γ=1.89806
  ^MvM7       : α=1.28208, β=1.49994, γ=1.87340
  ^MN7        : α=1.28208, β=1.49994, γ=1.84906

...

Half-diminished family (ø7 variants):
  øSM7        : α=1.20093, β=1.40500, γ=1.94837
  ø^M7        : α=1.20093, β=1.40500, γ=1.92305
  ømaj7       : α=1.20093, β=1.40500, γ=1.89806
  øvM7        

In [203]:
# ============================================================================
# 12-TET CHORD DEFINITIONS - FROM Harmonic_Eigenspace.js get12TETChordPositions()
# ============================================================================

def get_12tet_chord_positions() -> List[Tuple[str, float, float, float]]:
    """
    Get standard 12-TET chord positions - direct port from JS.
    
    Returns:
        List of (name, α, β, γ) tuples
    """
    return [
        ("maj7",    r12(4),  r12(7),  r12(11)),
        ("min7",    r12(3),  r12(7),  r12(10)),
        ("dom7",    r12(4),  r12(7),  r12(10)),
        ("mM7",     r12(3),  r12(7),  r12(11)),
        ("ø7",      r12(3),  r12(6),  r12(10)),
        ("dim7",    r12(3),  r12(6),  r12(9)),
        ("aug7",    r12(4),  r12(8),  r12(10)),
        ("augM7",   r12(4),  r12(8),  r12(11)),
        ("7sus4",   r12(5),  r12(7),  r12(10)),
        ("sus2",    r12(2),  r12(5),  r12(7)),
    ]


chords_12tet = get_12tet_chord_positions()
print(f"Total 12-TET reference chords: {len(chords_12tet)}")
print("\n12-TET chord positions:")
for name, alpha, beta, gamma in chords_12tet:
    print(f"  {name:10s}: α={alpha:.5f}, β={beta:.5f}, γ={gamma:.5f}")

Total 12-TET reference chords: 10

12-TET chord positions:
  maj7      : α=1.25992, β=1.49831, γ=1.88775
  min7      : α=1.18921, β=1.49831, γ=1.78180
  dom7      : α=1.25992, β=1.49831, γ=1.78180
  mM7       : α=1.18921, β=1.49831, γ=1.88775
  ø7        : α=1.18921, β=1.41421, γ=1.78180
  dim7      : α=1.18921, β=1.41421, γ=1.68179
  aug7      : α=1.25992, β=1.58740, γ=1.78180
  augM7     : α=1.25992, β=1.58740, γ=1.88775
  7sus4     : α=1.33484, β=1.49831, γ=1.78180
  sus2      : α=1.12246, β=1.33484, γ=1.49831


## Load Pre-computed Dissonance Map

The EigenSpace dissonance map was pre-computed at 220 Hz with:
- 150³ = 3,375,000 points  
- α, β, γ range: 1.0 to 2.0
- Plomp-Levelt dissonance model with 6 harmonics

In [204]:
# ============================================================================
# LOAD PRE-COMPUTED DISSONANCE MAP
# ============================================================================

def load_precomputed_dissonance_map(
    base_freq: int = 220,
    n_points: int = 150,
    r_low: float = 1.0,
    r_high: float = 2.0,
    dataset_path: str = "../dataset/EigenSpace_Data"
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Load pre-computed dissonance map from binary chunk files.
    
    Args:
        base_freq: Base frequency the map was computed at (Hz)
        n_points: Resolution (n³ grid points)
        r_low, r_high: Ratio range
        dataset_path: Path to EigenSpace_Data folder
        
    Returns:
        (alpha_range, beta_range, gamma_range, dissonance_3d)
    """
    print(f"Loading {n_points}³ pre-computed dataset from {dataset_path}...")
    
    # Find all chunk files
    chunk_files = sorted([
        f for f in os.listdir(dataset_path)
        if f.startswith(f"harmonic-{base_freq}Hz-{n_points}nodes-chunk")
    ])
    
    if not chunk_files:
        raise FileNotFoundError(
            f"No dataset chunks found for {base_freq}Hz, {n_points} nodes"
        )
    
    # Load all chunks
    all_data = []
    for chunk_file in chunk_files:
        chunk_path = os.path.join(dataset_path, chunk_file)
        chunk_data = np.fromfile(chunk_path, dtype=np.float32)
        all_data.append(chunk_data)
        print(f"  ✓ {chunk_file}: {len(chunk_data):,} values")
    
    # Concatenate and reshape
    flat_data = np.concatenate(all_data)
    dissonance_3d = flat_data.reshape((n_points, n_points, n_points))
    
    # Recreate coordinate ranges
    alpha_range = np.linspace(r_low, r_high, n_points)
    beta_range = np.linspace(r_low, r_high, n_points)
    gamma_range = np.linspace(r_low, r_high, n_points)
    
    print(f"\n✓ Loaded shape: {dissonance_3d.shape}")
    print(f"✓ Dissonance range: {dissonance_3d.min():.4f} to {dissonance_3d.max():.4f}")
    
    return alpha_range, beta_range, gamma_range, dissonance_3d


# Load the dataset
alpha_range, beta_range, gamma_range, dissonance_3d = load_precomputed_dissonance_map(
    base_freq=220,
    n_points=150
)

Loading 150³ pre-computed dataset from ../dataset/EigenSpace_Data...
  ✓ harmonic-220Hz-150nodes-chunk001.bin: 1,000,000 values
  ✓ harmonic-220Hz-150nodes-chunk002.bin: 1,000,000 values
  ✓ harmonic-220Hz-150nodes-chunk003.bin: 1,000,000 values
  ✓ harmonic-220Hz-150nodes-chunk004.bin: 375,000 values

✓ Loaded shape: (150, 150, 150)
✓ Dissonance range: 0.6725 to 30.6237


In [205]:
# ============================================================================
# DISSONANCE COMPUTATION — Direct Plomp-Levelt (exact, per-root frequency)
# ============================================================================
#
# The Plomp-Levelt critical bandwidth parameter S = Dstar / (S1*Fmin + S2)
# depends on ABSOLUTE frequency. The dissonance landscape varies with the
# root pitch — low-register chords have wider critical bands = more dissonance.
#
# EigenSpace (α, β, γ) are ratio vectors from the root. The 4th coordinate D
# must be computed from the ACTUAL root frequency of each chord, not a fixed
# reference. Each chord's root is the origin of its own dissonance vector.

def dissmeasure(fvec, amp, model="min"):
    """
    Plomp-Levelt dissonance measure — exact port from Harmonic_Eigenspace.js.
    
    Args:
        fvec: List of frequencies (Hz)
        amp: List of amplitudes
        model: "min" uses min(a_i, a_j) for pair weighting
        
    Returns:
        Total dissonance (float)
    """
    sorted_pairs = sorted(zip(fvec, amp), key=lambda x: x[0])
    fr = [p[0] for p in sorted_pairs]
    am = [p[1] for p in sorted_pairs]
    
    Dstar = 0.24
    S1, S2 = 0.0207, 18.96
    C1, C2 = 5.0, -5.0
    A1, A2 = -3.51, -5.75
    
    total = 0.0
    for i in range(len(fr)):
        for j in range(i + 1, len(fr)):
            Fmin = fr[i]
            S = Dstar / (S1 * Fmin + S2)
            Fdif = fr[j] - fr[i]
            a = min(am[i], am[j])
            SFdif = S * Fdif
            total += a * (C1 * np.exp(A1 * SFdif) + C2 * np.exp(A2 * SFdif))
    return total


def step53_to_hz(step_53: int) -> float:
    """
    Convert an absolute 53-TET step number to frequency in Hz.
    
    Mapping: MIDI note 69 (A4) = 440 Hz.
    MIDI note n → 53-TET step = round(n * 53/12).
    Inverse: step → freq = 440 * 2^((step/53) - (69/12))
    """
    return 440.0 * (2.0 ** (step_53 / 53.0 - 69.0 / 12.0))


def compute_dissonance(alpha: float, beta: float, gamma: float,
                       base_freq: float = 220.0, num_harmonics: int = 6) -> float:
    """
    Compute Plomp-Levelt dissonance for a chord at a given root frequency.
    
    The (α, β, γ, D) tuple IS the EigenSpace 4D coordinate.
    D depends on the root pitch because critical bandwidth is frequency-dependent.
    
    Args:
        alpha: Frequency ratio of 2nd note to root
        beta:  Frequency ratio of 3rd note to root
        gamma: Frequency ratio of 4th note to root
        base_freq: ROOT frequency in Hz — must be the actual pitch of the chord root
        num_harmonics: Number of harmonics per voice (6 = standard)
        
    Returns:
        Dissonance D (float). Lower = more consonant.
    """
    freqs = []
    amps = []
    for h in range(1, num_harmonics + 1):
        freqs.extend([
            base_freq * h,
            base_freq * h * alpha,
            base_freq * h * beta,
            base_freq * h * gamma,
        ])
        amps.extend([1.0, 1.0, 1.0, 1.0])
    return dissmeasure(freqs, amps)


# --- Map lookup (kept for 3D visualization only) ---

def get_dissonance_at_point(
    alpha: float, beta: float, gamma: float,
    alpha_range: np.ndarray, beta_range: np.ndarray, gamma_range: np.ndarray,
    dissonance_3d: np.ndarray
) -> Optional[float]:
    """
    Get dissonance from pre-computed 150³ map (trilinear interpolation).
    ⚠ USE compute_dissonance() with actual root Hz for accurate queries.
    """
    if (alpha < alpha_range[0] or alpha > alpha_range[-1] or
        beta < beta_range[0] or beta > beta_range[-1] or
        gamma < gamma_range[0] or gamma > gamma_range[-1]):
        return None
    
    def find_bracket(val, arr):
        idx = np.searchsorted(arr, val) - 1
        idx = max(0, min(idx, len(arr) - 2))
        t = (val - arr[idx]) / (arr[idx + 1] - arr[idx])
        return idx, t
    
    i, ti = find_bracket(alpha, alpha_range)
    j, tj = find_bracket(beta, beta_range)
    k, tk = find_bracket(gamma, gamma_range)
    
    c000 = dissonance_3d[i, j, k]
    c100 = dissonance_3d[i+1, j, k]
    c010 = dissonance_3d[i, j+1, k]
    c110 = dissonance_3d[i+1, j+1, k]
    c001 = dissonance_3d[i, j, k+1]
    c101 = dissonance_3d[i+1, j, k+1]
    c011 = dissonance_3d[i, j+1, k+1]
    c111 = dissonance_3d[i+1, j+1, k+1]
    
    c00 = c000 * (1 - ti) + c100 * ti
    c10 = c010 * (1 - ti) + c110 * ti
    c01 = c001 * (1 - ti) + c101 * ti
    c11 = c011 * (1 - ti) + c111 * ti
    c0 = c00 * (1 - tj) + c10 * tj
    c1 = c01 * (1 - tj) + c11 * tj
    return c0 * (1 - tk) + c1 * tk


# --- Demonstrate frequency dependence ---
print("Plomp-Levelt frequency dependence: same ratios, different root pitch")
print("=" * 80)
print(f"  {'Chord':<14} {'Root Hz':>8}  {'α':>7} {'β':>7} {'γ':>7}  {'D':>8}")
print("-" * 80)
for label, a, b, g in [
    ("maj7 (53-TET)",  r(18), r(31), r(49)),
    ("m7   (53-TET)",  r(13), r(31), r(44)),
    ("dom7 (53-TET)",  r(18), r(31), r(44)),
]:
    for root_hz in [65, 130, 220, 261, 440, 523]:
        d = compute_dissonance(a, b, g, base_freq=root_hz)
        note_name = {65:"C2", 130:"C3", 220:"A3", 261:"C4", 440:"A4", 523:"C5"}[root_hz]
        print(f"  {label:<14} {note_name:>4} {root_hz:>3}  {a:>7.4f} {b:>7.4f} {g:>7.4f}  {d:>8.4f}")
    print()

print("✓ Same interval ratios, DIFFERENT dissonance at different registers")
print("  → Each chord must be computed from its actual root frequency")

Plomp-Levelt frequency dependence: same ratios, different root pitch
  Chord           Root Hz        α       β       γ         D
--------------------------------------------------------------------------------
  maj7 (53-TET)    C2  65   1.2654  1.4999  1.8981   38.3907
  maj7 (53-TET)    C3 130   1.2654  1.4999  1.8981   20.2860
  maj7 (53-TET)    A3 220   1.2654  1.4999  1.8981   13.5842
  maj7 (53-TET)    C4 261   1.2654  1.4999  1.8981   12.2002
  maj7 (53-TET)    A4 440   1.2654  1.4999  1.8981    9.4351
  maj7 (53-TET)    C5 523   1.2654  1.4999  1.8981    8.8537

  m7   (53-TET)    C2  65   1.1853  1.4999  1.7779   40.8080
  m7   (53-TET)    C3 130   1.1853  1.4999  1.7779   21.1142
  m7   (53-TET)    A3 220   1.1853  1.4999  1.7779   13.2874
  m7   (53-TET)    C4 261   1.1853  1.4999  1.7779   11.6115
  m7   (53-TET)    A4 440   1.1853  1.4999  1.7779    8.2334
  m7   (53-TET)    C5 523   1.1853  1.4999  1.7779    7.5357

  dom7 (53-TET)    C2  65   1.2654  1.4999  1.7779   42

In [206]:
# ============================================================================
# DIAGNOSTIC: Investigate axis ordering bug
# ============================================================================
import itertools

print("=" * 70)
print("DIAGNOSTIC: Axis ordering and data integrity check")
print("=" * 70)

# 1. Check raw data at boundaries
print("\n--- 1. Corner values (raw array access) ---")
print(f"dissonance_3d[0,0,0]       = {dissonance_3d[0,0,0]:.4f}   (all axes=1.0, unison)")
print(f"dissonance_3d[149,149,149] = {dissonance_3d[149,149,149]:.4f}   (all axes=2.0, octave)")
print(f"dissonance_3d[0,0,149]     = {dissonance_3d[0,0,149]:.4f}")
print(f"dissonance_3d[149,0,0]     = {dissonance_3d[149,0,0]:.4f}")
print(f"dissonance_3d[0,149,0]     = {dissonance_3d[0,149,0]:.4f}")
print(f"\nNaN count: {np.isnan(dissonance_3d).sum():,} out of {dissonance_3d.size:,}")
print(f"Data range: {dissonance_3d.min():.4f} to {dissonance_3d.max():.4f}")

# 2. Check symmetry (the dissonance function IS symmetric in α,β,γ)
print("\n--- 2. Symmetry check ---")
idx_a, idx_b, idx_c = 20, 80, 120
perms_check = list(itertools.permutations([idx_a, idx_b, idx_c]))
for p in perms_check:
    print(f"  d3d[{p[0]:3d},{p[1]:3d},{p[2]:3d}] = {dissonance_3d[p[0],p[1],p[2]]:.6f}")

# 3. Compute dissonance directly for known chords and compare
print("\n--- 3. Direct computation vs map lookup for known chords ---")

def dissmeasure_py(fvec, amp, model="min"):
    sorted_pairs = sorted(zip(fvec, amp), key=lambda x: x[0])
    fr_sorted = [p[0] for p in sorted_pairs]
    am_sorted = [p[1] for p in sorted_pairs]
    Dstar, S1, S2 = 0.24, 0.0207, 18.96
    C1, C2 = 5, -5
    A1, A2 = -3.51, -5.75
    total = 0
    for i in range(len(fr_sorted)):
        for j in range(i+1, len(fr_sorted)):
            Fmin = fr_sorted[i]
            S = Dstar / (S1 * Fmin + S2)
            Fdif = fr_sorted[j] - fr_sorted[i]
            a = min(am_sorted[i], am_sorted[j])
            SFdif = S * Fdif
            total += a * (C1 * np.exp(A1 * SFdif) + C2 * np.exp(A2 * SFdif))
    return total

def compute_chord_dissonance(alpha, beta, gamma, base_freq=220, num_harmonics=6):
    freqs = []
    amps = []
    for h in range(1, num_harmonics + 1):
        freqs.extend([base_freq * h, base_freq * h * alpha,
                       base_freq * h * beta, base_freq * h * gamma])
        amps.extend([1.0, 1.0, 1.0, 1.0])
    return dissmeasure_py(freqs, amps)

test_chords = [
    ("12-TET maj7",     r12(4), r12(7), r12(11)),
    ("12-TET min7",     r12(3), r12(7), r12(10)),
    ("12-TET dom7",     r12(4), r12(7), r12(10)),
    ("53-TET maj7",     r(18),  r(31),  r(49)),
    ("53-TET m7",       r(13),  r(31),  r(44)),
    ("JI maj triad+7",  5/4,    3/2,    15/8),
    ("JI min triad+m7", 6/5,    3/2,    9/5),
    ("unison",          1.001,  1.001,  1.001),
    ("octave triple",   1.999,  1.999,  1.999),
    ("perf5 trio",      1.5,    1.5,    1.5),
]

print(f"\n{'Chord':<22} {'α':>7} {'β':>7} {'γ':>7} {'Map val':>9} {'Direct':>9} {'Match?':>7}")
print("-" * 75)
for name, a, b, g in test_chords:
    map_val = get_dissonance_at_point(a, b, g, alpha_range, beta_range, gamma_range, dissonance_3d)
    direct_val = compute_chord_dissonance(a, b, g)
    match = "✓" if map_val and abs(map_val - direct_val) < 0.5 else "✗"
    map_str = f"{map_val:.4f}" if map_val else "None"
    print(f"{name:<22} {a:>7.4f} {b:>7.4f} {g:>7.4f} {map_str:>9} {direct_val:>9.4f} {match:>7}")

# 4. What num_harmonics was the data computed with?
# Test by computing at a known grid point with different harmonic counts
print("\n--- 4. Detecting computation parameters ---")
grid_idx = 75  # middle of grid
grid_val = alpha_range[grid_idx]
grid_diss = dissonance_3d[grid_idx, grid_idx, grid_idx]
print(f"Grid point [{grid_idx},{grid_idx},{grid_idx}] = ({grid_val:.4f}, {grid_val:.4f}, {grid_val:.4f}), stored value = {grid_diss:.6f}")

for n_harm in [4, 5, 6, 7, 8]:
    for base_f in [220, 500]:
        computed = compute_chord_dissonance(grid_val, grid_val, grid_val, base_freq=base_f, num_harmonics=n_harm)
        match = "← MATCH" if abs(computed - grid_diss) < 0.05 else ""
        print(f"  {base_f}Hz, {n_harm} harmonics: {computed:.6f} {match}")

# 5. Try ALL 6 axis permutations — which gives best match?
print("\n--- 5. Test ALL axis permutations ---")
perms = list(itertools.permutations([0, 1, 2]))
perm_labels = ["α,β,γ", "α,γ,β", "β,α,γ", "β,γ,α", "γ,α,β", "γ,β,α"]

for perm, label in zip(perms, perm_labels):
    total_err = 0
    n_valid = 0
    for name, a, b, g in test_chords[:7]:
        coords = [a, b, g]
        # Map coordinates to array indices via permutation
        # perm[0]=which coordinate maps to axis0, etc.
        i0 = np.searchsorted(alpha_range, coords[perm[0]]) - 1
        i1 = np.searchsorted(beta_range,  coords[perm[1]]) - 1
        i2 = np.searchsorted(gamma_range, coords[perm[2]]) - 1
        i0 = max(0, min(i0, 148))
        i1 = max(0, min(i1, 148))
        i2 = max(0, min(i2, 148))
        
        map_val = dissonance_3d[i0, i1, i2]
        direct_val = compute_chord_dissonance(a, b, g)
        if not np.isnan(map_val):
            total_err += abs(map_val - direct_val)
            n_valid += 1
    
    avg_err = total_err / n_valid if n_valid > 0 else 999
    print(f"  axes=[{label}]: avg |error| = {avg_err:.4f}")


DIAGNOSTIC: Axis ordering and data integrity check

--- 1. Corner values (raw array access) ---
dissonance_3d[0,0,0]       = 1.8189   (all axes=1.0, unison)
dissonance_3d[149,149,149] = 0.6725   (all axes=2.0, octave)
dissonance_3d[0,0,149]     = 1.3933
dissonance_3d[149,0,0]     = 1.3933
dissonance_3d[0,149,0]     = 1.3933

NaN count: 0 out of 3,375,000
Data range: 0.6725 to 30.6237

--- 2. Symmetry check ---
  d3d[ 20, 80,120] = 17.096001
  d3d[ 20,120, 80] = 17.096001
  d3d[ 80, 20,120] = 17.096001
  d3d[ 80,120, 20] = 17.096001
  d3d[120, 20, 80] = 17.096001
  d3d[120, 80, 20] = 17.096001

--- 3. Direct computation vs map lookup for known chords ---

Chord                        α       β       γ   Map val    Direct  Match?
---------------------------------------------------------------------------
12-TET maj7             1.2599  1.4983  1.8877   13.7181   13.4394       ✓
12-TET min7             1.1892  1.4983  1.7818   13.4701   13.1944       ✓
12-TET dom7             1.2599  1.49

In [207]:
# ============================================================================
# MAP ALL CHORDS WITH DISSONANCE VALUES (using direct Plomp-Levelt)
# ============================================================================

def map_chords_to_eigenspace(
    chords: List[Tuple[str, float, float, float]],
    system_name: str = "53-TET"
) -> pd.DataFrame:
    """
    Map chord positions to EigenSpace and compute exact dissonance values.
    
    Uses direct Plomp-Levelt computation (not the pre-computed 150³ map).
    
    Args:
        chords: List of (name, α, β, γ) tuples
        system_name: Name of the tuning system
        
    Returns:
        DataFrame with chord positions and dissonance values
    """
    records = []
    
    for name, alpha, beta, gamma in chords:
        in_tetrahedron = alpha <= beta <= gamma
        
        # Direct computation — exact at any ratio
        diss = compute_dissonance(alpha, beta, gamma)
        
        alpha_cents = 1200 * np.log2(alpha) if alpha > 0 else 0
        beta_cents = 1200 * np.log2(beta) if beta > 0 else 0
        gamma_cents = 1200 * np.log2(gamma) if gamma > 0 else 0
        
        records.append({
            'name': name,
            'system': system_name,
            'alpha': alpha,
            'beta': beta,
            'gamma': gamma,
            'alpha_cents': alpha_cents,
            'beta_cents': beta_cents,
            'gamma_cents': gamma_cents,
            'dissonance': diss,
            'in_tetrahedron': in_tetrahedron
        })
    
    return pd.DataFrame(records)


# Map both 53-TET and 12-TET chords
df_53tet = map_chords_to_eigenspace(chords_53tet, "53-TET")
df_12tet = map_chords_to_eigenspace(chords_12tet, "12-TET")

df_all_chords = pd.concat([df_53tet, df_12tet], ignore_index=True)

print(f"✓ Mapped {len(df_53tet)} 53-TET chords (direct Plomp-Levelt)")
print(f"✓ Mapped {len(df_12tet)} 12-TET chords")
print(f"✓ Total: {len(df_all_chords)} chords\n")

print("Dissonance Statistics by System:")
print(df_all_chords.groupby('system')['dissonance'].describe())

# Show top-10 most consonant 53-TET chords
print("\n\nTop 10 most consonant 53-TET chords:")
top10 = df_53tet.nsmallest(10, 'dissonance')
for _, row in top10.iterrows():
    print(f"  {row['name']:12s}  D={row['dissonance']:.4f}  α={row['alpha']:.5f} β={row['beta']:.5f} γ={row['gamma']:.5f}")

✓ Mapped 168 53-TET chords (direct Plomp-Levelt)
✓ Mapped 10 12-TET chords
✓ Total: 178 chords

Dissonance Statistics by System:
        count       mean       std        min        25%        50%  \
system                                                                
12-TET   10.0  14.967332  1.328155  13.194435  13.762464  15.149229   
53-TET  168.0  15.152347  1.531086   9.829594  14.327806  15.223608   

              75%        max  
system                        
12-TET  15.779518  16.742661  
53-TET  16.263495  18.146794  


Top 10 most consonant 53-TET chords:
  vMP           D=9.8296  α=1.24898 β=1.49994 γ=1.49994
  ^mP           D=10.0397  α=1.20093 β=1.49994 γ=1.49994
  sus4P         D=10.9039  α=1.33339 β=1.49994 γ=1.49994
  MP            D=11.4470  α=1.26543 β=1.49994 γ=1.49994
  ^m^m7         D=11.5871  α=1.20093 β=1.49994 γ=1.80132
  mP            D=11.7324  α=1.18533 β=1.49994 γ=1.49994
  NP            D=11.7448  α=1.23276 β=1.49994 γ=1.49994
  nP            D=11.7987

## Test Case: "Something" (Beatles) — MIDI Data → EigenSpace

Parse the actual 53-TET MPE MIDI file for "Something" (type_0_major variant).  
For each chord: extract notes from MIDI+pitch bend, normalize to a single octave,
identify the third/fifth/seventh intervals, compute (α, β, γ) ratios, and look up dissonance.

**Rules:**
- Normalize all notes to one octave relative to root (mod 53)
- Discard 9th and higher extensions (keep only intervals < 53 steps)
- Map intervals to third / fifth / seventh bins
- For triads (no 7th), set γ = 2.0 (octave doubling)
- Compute dissonance via trilinear interpolation in the pre-computed map

In [208]:
# ============================================================================
# PARSE MPE MIDI → ONE-OCTAVE PITCH CLASSES (the EigenSpace foundation)
# ============================================================================
#
# ┌──────────────────────────────────────────────────────────────────────────┐
# │  EIGENSPACE PRINCIPLE                                                    │
# │                                                                          │
# │  The EigenSpace maps ALL possible relations of THREE notes relative to   │
# │  a FUNDAMENTAL. All four notes (root + 3 chord tones) must lie within    │
# │  ONE OCTAVE: frequency ratios in (1.0, 2.0].                            │
# │                                                                          │
# │  Axes:  α = 2nd chord tone    (e.g. third or sus2/sus4)                 │
# │         β = 3rd chord tone    (e.g. fifth)                              │
# │         γ = 4th chord tone    (e.g. seventh, or 2.0 for triads)         │
# │                                                                          │
# │  Constraint: 1.0 < α ≤ β ≤ γ ≤ 2.0  (tetrahedron ordering)            │
# │                                                                          │
# │  VOICING IS STRIPPED. Notes from the MIDI spread across 2-3 octaves     │
# │  (drop-2, open, shell voicings). We fold ALL notes to one octave via    │
# │  modulo 53:  pitch_class = (absolute_step - root_step) % 53             │
# │                                                                          │
# │  After folding, we get the unique PITCH CLASSES of the chord.           │
# │  Extensions (9th, 11th, 13th) that fold to 2nd, 4th, 6th territory     │
# │  are DISCARDED in the next step — only the tetrad core matters.         │
# └──────────────────────────────────────────────────────────────────────────┘

import mido

MIDI_PATH = "../dataset/midi_files/53_tet_mpe/47832_Something_C_major_type_0_major.mid"


def midi_bend_to_53tet_step(midi_note: int, pitch_bend: int) -> int:
    """Convert MIDI note + pitch bend (±2 semitone range) to absolute 53-TET step."""
    bend_semitones = (pitch_bend / 8192) * 2  # ±2 semitones MPE range
    true_semitone = midi_note + bend_semitones
    return round(true_semitone * (53 / 12))


def parse_mpe_midi_chords(midi_path: str) -> List[Dict]:
    """
    Parse an MPE MIDI file → one-octave pitch classes per chord.
    
    The voicing system (voicing.py) spreads notes across 2–3 octaves using
    drop-2, open, shell, and extended voicings. For EigenSpace mapping, we
    must UNDO the voicing and recover the abstract pitch-class content.
    
    Process:
        1. Read MIDI note-on events grouped by onset time
        2. Convert each note (MIDI note + pitch bend) → absolute 53-TET step
        3. Identify root = lowest note → root frequency in Hz
        4. Compute intervals: (step - root) % 53  (fold to one octave)
        5. Deduplicate → unique PITCH CLASSES within one octave (0–52 steps)
    
    The resulting intervals are OCTAVE-NORMALIZED: every note is expressed
    as a ratio between 1.0 and 2.0 relative to the root. This is exactly
    what EigenSpace needs.
    
    Returns list of dicts:
        {
            'beat':      float,           # onset in beats
            'steps_53':  List[int],       # absolute 53-TET steps (raw voiced notes)
            'root_step': int,             # absolute 53-TET step of root (lowest)
            'root_hz':   float,           # root frequency in Hz
            'intervals': List[int],       # ONE-OCTAVE pitch classes (0–52, sorted, unique)
        }
    """
    mid = mido.MidiFile(midi_path)
    tpb = mid.ticks_per_beat
    track = mid.tracks[1]  # notes are always on track 1

    # Collect all timestamped events
    ticks = 0
    events = []
    for msg in track:
        ticks += msg.time
        if not msg.is_meta:
            events.append((ticks, msg))

    # Group note_on events by onset tick
    channel_bend = {}
    chords_raw = []
    current_tick = None
    current_notes = []

    for tick, msg in events:
        if msg.type == 'pitchwheel':
            channel_bend[msg.channel] = msg.pitch
        elif msg.type == 'note_on' and msg.velocity > 0:
            bend = channel_bend.get(msg.channel, 0)
            if current_tick is None or tick != current_tick:
                if current_notes:
                    chords_raw.append((current_tick, list(current_notes)))
                current_notes = []
                current_tick = tick
            current_notes.append((msg.note, bend))

    if current_notes:
        chords_raw.append((current_tick, list(current_notes)))

    # ── Convert to one-octave pitch classes ────────────────────────────────
    chords = []
    for tick, notes in chords_raw:
        steps = sorted([midi_bend_to_53tet_step(n, b) for n, b in notes])
        root_step = min(steps)
        root_hz = step53_to_hz(root_step)
        
        # OCTAVE NORMALIZATION: fold all notes to one octave via % 53
        # This strips the voicing and gives us pure pitch classes.
        # Duplicated pitch classes (e.g. root doubled at octave) collapse.
        intervals = sorted(set((s - root_step) % 53 for s in steps))
        
        chords.append({
            'beat': tick / tpb,
            'steps_53': steps,
            'root_step': root_step,
            'root_hz': root_hz,
            'intervals': intervals,
        })

    return chords


# ── Parse "Something" ────────────────────────────────────────────────────
raw_chords = parse_mpe_midi_chords(MIDI_PATH)
song_name = "Something"

print(f"Parsed {len(raw_chords)} chords from MIDI")
print(f"  Song: '{song_name}'")
print(f"  File: {os.path.basename(MIDI_PATH)}")
print()

# Show first 15 with both raw voicing and one-octave intervals
print(f"  {'#':>3} {'Beat':>6}  {'Root':>5} {'Root Hz':>8}  {'Raw voiced (abs steps)':42s}  One-octave intervals")
print("-" * 120)
for i, ch in enumerate(raw_chords[:15]):
    raw_ivs = sorted([s - ch['root_step'] for s in ch['steps_53']])
    print(f"  {i+1:3d} {ch['beat']:6.1f}  {ch['root_step']:>5d} {ch['root_hz']:>8.1f}  {str(raw_ivs):42s}  {ch['intervals']}")

# Show root frequency range
root_freqs = [ch['root_hz'] for ch in raw_chords]
print(f"\nRoot frequency range: {min(root_freqs):.1f} Hz – {max(root_freqs):.1f} Hz")
print(f"  ({1200 * np.log2(max(root_freqs)/min(root_freqs)):.0f} cents span)")
print(f"\nVoicing spans: {min(max(ch['steps_53'])-min(ch['steps_53']) for ch in raw_chords)}"
      f"–{max(max(ch['steps_53'])-min(ch['steps_53']) for ch in raw_chords)} steps"
      f" → ALL folded to 0–52 (one octave)")

Parsed 165 chords from MIDI
  Song: 'Something'
  File: 47832_Something_C_major_type_0_major.mid

    #   Beat   Root  Root Hz  Raw voiced (abs steps)                      One-octave intervals
------------------------------------------------------------------------------------------------------------------------
    1    0.0    234    174.4  [0, 31, 53, 71, 106]                        [0, 18, 31]
    2    4.0    226    157.1  [0, 30, 53, 70, 106]                        [0, 17, 30]
    3    6.0    221    147.2  [0, 22, 40, 66, 75]                         [0, 13, 22, 40]
    4    8.0    212    130.8  [0, 31, 53, 71, 106]                        [0, 18, 31]
    5   16.0    212    130.8  [0, 31, 53, 71, 102]                        [0, 18, 31, 49]
    6   24.0    212    130.8  [0, 31, 53, 71, 97]                         [0, 18, 31, 44]
    7   32.0    181     87.2  [0, 53, 84, 106, 124]                       [0, 18, 31]
    8   38.0    177     82.8  [0, 57, 57, 88, 110, 128]                 

In [209]:
# ============================================================================
# TETRAD EXTRACTION: Pitch classes → EigenSpace (α, β, γ)
# ============================================================================
#
# ┌──────────────────────────────────────────────────────────────────────────┐
# │  EIGENSPACE MAPPING — THE PRINCIPLE                                      │
# │                                                                          │
# │  From the one-octave pitch classes, we extract the TETRAD:               │
# │  Root + at most 3 chord tones that define (α, β, γ).                    │
# │                                                                          │
# │  A chord = root frequency × {1, α, β, γ}  where 1 < α ≤ β ≤ γ ≤ 2     │
# │                                                                          │
# │  The three tones map to three DISJOINT frequency bands:                  │
# │                                                                          │
# │    α  SECOND/THIRD zone  steps 10–22   (227–498¢)                       │
# │       ▸ thirds (sm=11 to SM=20), sus4 (22)                              │
# │       ▸ in 53-TET vocabulary: {11,12,13,14,15,16,17,18,19,20,22}        │
# │                                                                          │
# │    β  FIFTH zone          steps 24–35   (543–792¢)                      │
# │       ▸ dim/perfect/augmented fifths                                     │
# │       ▸ in 53-TET vocabulary: {26, 31, 35}                              │
# │                                                                          │
# │    γ  SEVENTH zone        steps 42–52   (951–1177¢)                     │
# │       ▸ all seventh qualities (sm7=42 to SM7=51)                         │
# │       ▸ in 53-TET vocabulary: {42,43,44,45,46,47,48,49,50,51}           │
# │                                                                          │
# │  WHAT IS DISCARDED (→ extras, NOT mapped):                               │
# │    • steps 1–9    seconds / 9th extensions  (23–204¢)                   │
# │    • steps 36–41  sixths / 13th extensions  (815–928¢)                  │
# │    • steps 23     no-man's-land between 3rd and 5th zones               │
# │                                                                          │
# │  Each slot accepts AT MOST ONE note. If multiple pitch classes fall      │
# │  in the same zone, the LOWEST (smallest interval) wins — this gives     │
# │  priority to the core chord tone over folded extensions.                 │
# │  (e.g. minor 3rd at 13 wins over sus4/11th at 22)                       │
# │                                                                          │
# │  After selection, each step is SNAPPED to its nearest CANONICAL          │
# │  vocabulary value. This ensures the chord lands exactly on a             │
# │  53-TET vocabulary grid position in EigenSpace.                          │
# └──────────────────────────────────────────────────────────────────────────┘

# ── Canonical vocabulary values ──────────────────────────────────────────
# These are the ONLY step values that appear in get_53tet_chord_positions().
CANONICAL_THIRDS   = sorted([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22])
CANONICAL_FIFTHS   = sorted([26, 31, 35])
CANONICAL_SEVENTHS = sorted([42, 43, 44, 45, 46, 47, 48, 49, 50, 51])

# ── Classification zones (one note per zone, disjoint, exhaustive) ───────
#   0       = root (always removed)
#   1–9     = seconds / 9ths      → discarded (extensions)
#   10–22   = α zone (thirds)     → mapped to α axis
#   23      = gap                 → discarded
#   24–35   = β zone (fifths)     → mapped to β axis
#   36–41   = sixths / 13ths      → discarded (extensions)
#   42–52   = γ zone (sevenths)   → mapped to γ axis
THIRD_RANGE   = range(10, 23)   # 10–22 steps
FIFTH_RANGE   = range(24, 36)   # 24–35 steps
SEVENTH_RANGE = range(42, 53)   # 42–52 steps


def _snap_to_nearest(value: int, canonical: List[int]) -> int:
    """Snap a step value to the nearest canonical vocabulary value."""
    return min(canonical, key=lambda c: abs(c - value))


def classify_chord_intervals(intervals: List[int]) -> Dict:
    """
    Extract the tetrad from one-octave pitch classes → EigenSpace (α, β, γ).
    
    This is the critical mapping function. From all pitch classes present
    in the chord (after octave normalization), it selects AT MOST 3 notes:
    one per EigenSpace axis (α, β, γ).
    
    Every note outside the three zones is discarded as an extension.
    Every selected note is snapped to the nearest canonical vocabulary step.
    
    The result: ratios α, β, γ ∈ (1.0, 2.0] — the chord's position in
    the EigenSpace, independent of voicing, octave, or root pitch.
    
    Args:
        intervals: One-octave pitch classes from parse_mpe_midi_chords()
                   (sorted, unique, 0 = root, all values 0–52)
    
    Returns:
        dict with: third, fifth, seventh (canonical steps or None),
                   alpha, beta, gamma (frequency ratios),
                   chord_type (label), is_triad, extras
    """
    # Filter out root (0) — already in one octave
    iv = sorted([x for x in intervals if 0 < x < 53])
    
    # ── Assign one note per zone (lowest pitch wins) ──
    third = None      # α axis — first note in 10–22
    fifth = None      # β axis — first note in 24–35
    seventh = None    # γ axis — first note in 42–52
    extras = []       # everything else: extensions, duplicates, gaps
    
    for step in iv:
        if step in THIRD_RANGE and third is None:
            third = step
        elif step in FIFTH_RANGE and fifth is None:
            fifth = step
        elif step in SEVENTH_RANGE and seventh is None:
            seventh = step
        else:
            extras.append(step)
    
    # ── Snap to canonical vocabulary values ──
    if third is not None:
        third = _snap_to_nearest(third, CANONICAL_THIRDS)
    if fifth is not None:
        fifth = _snap_to_nearest(fifth, CANONICAL_FIFTHS)
    else:
        # No fifth detected → default to perfect fifth (31)
        # Every vocabulary chord has an explicit fifth.
        fifth = 31
    if seventh is not None:
        seventh = _snap_to_nearest(seventh, CANONICAL_SEVENTHS)
    
    # ── Build chord type label ──
    THIRD_NAMES = {
        11: "sm", 12: "vm", 13: "m", 14: "^m", 15: "n",
        16: "N", 17: "vM", 18: "M", 19: "^M", 20: "SM", 22: "sus4"
    }
    FIFTH_NAMES = {26: "ø", 31: "P", 35: "+"}
    SEVENTH_NAMES = {
        42: "sm7", 43: "vm7", 44: "m7", 45: "^m7",
        46: "n7", 47: "N7", 48: "vM7", 49: "M7", 50: "^M7", 51: "SM7"
    }
    
    t_name = THIRD_NAMES.get(third, f"?{third}") if third else "–"
    f_name = FIFTH_NAMES.get(fifth, f"?{fifth}")
    s_name = SEVENTH_NAMES.get(seventh, f"?{seventh}") if seventh else "–"
    chord_type = f"{t_name}-{f_name}-{s_name}"
    
    # ── Compute frequency ratios (all within one octave: 1.0 – 2.0) ──
    # For triads (no seventh): γ = β  — the seventh collapses to the fifth.
    # This places triads on the γ=β diagonal of the tetrahedron.
    alpha = get_53tet_ratio(third) if third else 1.0   # no third → unison
    beta  = get_53tet_ratio(fifth)                      # always present
    gamma = get_53tet_ratio(seventh) if seventh else beta  # triad → γ = β
    
    # ── Validate: all ratios must be in (1.0, 2.0] ──
    assert 1.0 <= alpha <= 2.0, f"α={alpha} out of octave range (third={third})"
    assert 1.0 <  beta  <= 2.0, f"β={beta} out of octave range (fifth={fifth})"
    assert 1.0 <  gamma <= 2.0, f"γ={gamma} out of octave range (seventh={seventh})"
    
    is_triad = seventh is None
    
    return {
        'third': third,
        'fifth': fifth,
        'seventh': seventh,
        'alpha': alpha,
        'beta': beta,
        'gamma': gamma,
        'chord_type': chord_type,
        'is_triad': is_triad,
        'extras': extras,
    }


# ═══════════════════════════════════════════════════════════════════════════
# VALIDATION
# ═══════════════════════════════════════════════════════════════════════════

print("TETRAD EXTRACTION — Classification test")
print("=" * 120)
print(f"  {'#':>3} {'Beat':>5}  {'All pitch classes':24s}  {'→ Tetrad':16s}  "
      f"{'α':>8} {'β':>8} {'γ':>8}  {'Type':>5}  {'Extras'}")
print("-" * 120)

for i, ch in enumerate(raw_chords[:12]):
    cl = classify_chord_intervals(ch['intervals'])
    status = "TRI" if cl['is_triad'] else "TET"
    extra_str = str(cl['extras']) if cl['extras'] else "–"
    tetrad = f"[{cl['third'] or '–'}, {cl['fifth']}, {cl['seventh'] or '–'}]"
    print(
        f"  {i+1:3d} {ch['beat']:5.1f}  {str(ch['intervals']):24s}  {tetrad:16s}  "
        f"{cl['alpha']:8.5f} {cl['beta']:8.5f} {cl['gamma']:8.5f}  "
        f"{status:>5s}  {extra_str}"
    )

# ── Grid & octave validation ──
print(f"\n{'='*120}")
off_grid = 0
out_of_octave = 0
for ch in raw_chords:
    cl = classify_chord_intervals(ch['intervals'])
    if cl['third'] is not None and cl['third'] not in CANONICAL_THIRDS:
        off_grid += 1
    if cl['fifth'] not in CANONICAL_FIFTHS:
        off_grid += 1
    if cl['seventh'] is not None and cl['seventh'] not in CANONICAL_SEVENTHS:
        off_grid += 1
    if not (1.0 <= cl['alpha'] <= cl['beta'] <= cl['gamma'] <= 2.0):
        out_of_octave += 1

print(f"  Chords:              {len(raw_chords)}")
print(f"  Off-grid intervals:  {off_grid}")
print(f"  Outside tetrahedron: {out_of_octave}")
if off_grid == 0:
    print(f"  ✓ ALL chords on canonical 53-TET vocabulary grid")
if out_of_octave == 0:
    print(f"  ✓ ALL ratios satisfy 1.0 ≤ α ≤ β ≤ γ ≤ 2.0")

# ── Zone map ──
print(f"\n  EigenSpace zone map (53 steps = one octave):")
print(f"    0              root")
print(f"    1–9    (23–204¢)   ··· 2nds / 9ths ···  → discarded")
print(f"    10–22  (227–498¢)  ░░░ α THIRD zone ░░░  → {len(CANONICAL_THIRDS)} canonical values")
print(f"    23     (521¢)      ··· gap ···            → discarded")
print(f"    24–35  (543–792¢)  ▓▓▓ β FIFTH zone ▓▓▓  → {len(CANONICAL_FIFTHS)} canonical values")
print(f"    36–41  (815–928¢)  ··· 6ths / 13ths ···  → discarded")
print(f"    42–52  (951–1177¢) ███ γ SEVENTH zone ██  → {len(CANONICAL_SEVENTHS)} canonical values")

TETRAD EXTRACTION — Classification test
    #  Beat  All pitch classes         → Tetrad                 α        β        γ   Type  Extras
------------------------------------------------------------------------------------------------------------------------
    1   0.0  [0, 18, 31]               [18, 31, –]        1.26543  1.49994  1.49994    TRI  –
    2   4.0  [0, 17, 30]               [17, 31, –]        1.24898  1.49994  1.49994    TRI  –
    3   6.0  [0, 13, 22, 40]           [13, 31, –]        1.18533  1.49994  1.49994    TRI  [22, 40]
    4   8.0  [0, 18, 31]               [18, 31, –]        1.26543  1.49994  1.49994    TRI  –
    5  16.0  [0, 18, 31, 49]           [18, 31, 49]       1.26543  1.49994  1.89806    TET  –
    6  24.0  [0, 18, 31, 44]           [18, 31, 44]       1.26543  1.49994  1.77792    TET  –
    7  32.0  [0, 18, 31]               [18, 31, –]        1.26543  1.49994  1.49994    TRI  –
    8  38.0  [0, 4, 22, 35]            [22, 35, –]        1.33339  1.58050 

In [210]:
# ============================================================================
# MAP CHORD SEQUENCE → EigenSpace 4D (α, β, γ, D)
# ============================================================================
#
# For each chord in the song:
#   1. Extract tetrad from one-octave pitch classes  → (α, β, γ)
#   2. Compute Plomp-Levelt dissonance at the chord's ACTUAL root Hz → D
#
# The 4D coordinate (α, β, γ, D) is unique per chord:
#   • (α, β, γ) depends ONLY on the interval structure (voicing-independent)
#   • D depends on (α, β, γ) AND the root frequency (register-dependent)
#
# Two chords with identical intervals but different roots will share
# the same (α, β, γ) position but have different D values.

def map_song_to_eigenspace(chords: List[Dict]) -> pd.DataFrame:
    """
    Map a song's chords to EigenSpace 4D coordinates.
    
    Each chord's root defines the origin. The three axis ratios (α, β, γ)
    are computed from the tetrad pitch classes. Dissonance D is computed
    at the chord's actual root frequency via Plomp-Levelt.
    
    Args:
        chords: Output of parse_mpe_midi_chords()
        
    Returns:
        DataFrame with: beat, intervals, chord_type, alpha, beta, gamma,
        dissonance, root_hz, root_step, is_triad, in_tetrahedron, extras
    """
    rows = []
    for ch in chords:
        cl = classify_chord_intervals(ch['intervals'])
        
        in_tetrahedron = cl['alpha'] <= cl['beta'] <= cl['gamma']
        
        # Dissonance at THIS chord's actual root frequency
        diss = compute_dissonance(
            cl['alpha'], cl['beta'], cl['gamma'],
            base_freq=ch['root_hz']
        )
        
        rows.append({
            'beat': ch['beat'],
            'intervals': str(ch['intervals']),
            'chord_type': cl['chord_type'],
            'third': cl['third'],
            'fifth': cl['fifth'],
            'seventh': cl['seventh'],
            'alpha': cl['alpha'],
            'beta': cl['beta'],
            'gamma': cl['gamma'],
            'dissonance': diss,
            'root_step': ch['root_step'],
            'root_hz': ch['root_hz'],
            'is_triad': cl['is_triad'],
            'in_tetrahedron': in_tetrahedron,
            'extras': str(cl['extras']) if cl['extras'] else '',
        })
    
    return pd.DataFrame(rows)


# ── Map "Something" ──────────────────────────────────────────────────────
df_something = map_song_to_eigenspace(raw_chords)

print(f"'{song_name}' — {len(df_something)} chords mapped to EigenSpace 4D")
print(f"  Root Hz range: {df_something['root_hz'].min():.1f} – {df_something['root_hz'].max():.1f}")
print(f"  Triads:  {df_something['is_triad'].sum()}")
print(f"  Tetrads: {(~df_something['is_triad']).sum()}")
print(f"  D range: {df_something['dissonance'].min():.3f} – {df_something['dissonance'].max():.3f}")
print()

# Show unique chord types
unique_types = df_something.drop_duplicates(subset='chord_type').sort_values('dissonance')
print(f"Unique tetrad types: {len(unique_types)}")
print("=" * 120)
print(f"  {'Type':18s} {'3rd':>4s} {'5th':>4s} {'7th':>4s}   "
      f"{'α':>8s} {'β':>8s} {'γ':>8s}   {'D':>8s}  {'Root Hz':>8s}  {'Tet?':>4s}  Cnt")
print("-" * 120)

for _, row in unique_types.iterrows():
    count = (df_something['chord_type'] == row['chord_type']).sum()
    tet = "✓" if row['in_tetrahedron'] else "✗"
    print(
        f"  {row['chord_type']:18s} {row['third'] or '–':>4} {row['fifth'] or '–':>4} {row['seventh'] or '–':>4}   "
        f"{row['alpha']:8.5f} {row['beta']:8.5f} {row['gamma']:8.5f}   "
        f"{row['dissonance']:8.3f}  {row['root_hz']:8.1f}  {tet:>4s}  {count}"
    )

'Something' — 165 chords mapped to EigenSpace 4D
  Root Hz range: 65.4 – 174.4
  Triads:  117
  Tetrads: 48
  D range: 13.961 – 41.928

Unique tetrad types: 13
  Type                3rd  5th  7th          α        β        γ          D   Root Hz  Tet?  Cnt
------------------------------------------------------------------------------------------------------------------------
  vM-P-–               17   31  nan    1.24898  1.49994  1.49994     13.961     157.1     ✓  33
  M-P-–                18   31  nan    1.26543  1.49994  1.49994     14.074     174.4     ✓  36
  m-P-–                13   31  nan    1.18533  1.49994  1.49994     16.757     147.2     ✓  18
  M-P-M7               18   31 49.0    1.26543  1.49994  1.89806     20.179     130.8     ✓  6
  M-P-m7               18   31 44.0    1.26543  1.49994  1.77792     22.673     130.8     ✓  12
  m-ø-–                13   26  nan    1.18533  1.40500  1.40500     25.017     124.1     ✓  6
  sus4-P-–             22   31  nan    1.33339  

In [211]:
# ============================================================================
# FULL PROGRESSION TABLE — beat-by-beat with root frequency
# ============================================================================

print("'Something' (Beatles) — Full Chord Progression in EigenSpace")
print("=" * 130)
print(f"  {'#':>3s}  {'Beat':>6s}  {'Root Hz':>8s}  {'Intervals':24s}  {'Type':18s}  "
      f"{'α':>8s}  {'β':>8s}  {'γ':>8s}  {'D':>8s}  Ext")
print("-" * 130)

for i, row in df_something.iterrows():
    extras = row['extras'] if row['extras'] else ""
    marker = " ◄" if row['is_triad'] else ""
    print(
        f"  {i+1:3d}  {row['beat']:6.1f}  {row['root_hz']:8.1f}  {row['intervals']:24s}  {row['chord_type']:18s}  "
        f"{row['alpha']:8.5f}  {row['beta']:8.5f}  {row['gamma']:8.5f}  "
        f"{row['dissonance']:8.3f}  {extras}{marker}"
    )

'Something' (Beatles) — Full Chord Progression in EigenSpace
    #    Beat   Root Hz  Intervals                 Type                       α         β         γ         D  Ext
----------------------------------------------------------------------------------------------------------------------------------
    1     0.0     174.4  [0, 18, 31]               M-P-–                1.26543   1.49994   1.49994    14.074   ◄
    2     4.0     157.1  [0, 17, 30]               vM-P-–               1.24898   1.49994   1.49994    13.961   ◄
    3     6.0     147.2  [0, 13, 22, 40]           m-P-–                1.18533   1.49994   1.49994    16.757  [22, 40] ◄
    4     8.0     130.8  [0, 18, 31]               M-P-–                1.26543   1.49994   1.49994    18.566   ◄
    5    16.0     130.8  [0, 18, 31, 49]           M-P-M7               1.26543   1.49994   1.89806    20.179  
    6    24.0     130.8  [0, 18, 31, 44]           M-P-m7               1.26543   1.49994   1.77792    22.673  
    7

In [212]:
# ============================================================================
# 3D EIGENSPACE VISUALIZATION
# ============================================================================
# Replicates the visualization from Harmonic_Eigenspace.js:
#   - Color scale from the 150³ dissonance MAP (percentiles of the field)
#   - 53-TET chords as positioned dots, colored by their map dissonance
#   - "Something" chords overlaid, same color scale
#   - All dissonance at 220 Hz / 6 harmonics (the map reference)

# ── Color scale from the dissonance MAP ─────────────────────────────────────
# JS:  vmin = percentile(dData, 5);  vmax = percentile(dData, 95);
# dissonance_4D.ipynb:  vmin = np.percentile(dissonance_3d, 5);  vmax = np.percentile(dissonance_3d, 80)
# Both converge on ≈14.21 / ≈19.70 in the stored outputs. Using p5/p80 to match.
D_MAP_MIN = float(np.percentile(dissonance_3d, 5))
D_MAP_MAX = float(np.percentile(dissonance_3d, 80))

print(f"EigenSpace color scale (from 150³ map at 220 Hz):")
print(f"  cmin = {D_MAP_MIN:.4f}  (percentile 5)")
print(f"  cmax = {D_MAP_MAX:.4f}  (percentile 80)")
print(f"  Map absolute range: {dissonance_3d.min():.4f} – {dissonance_3d.max():.4f}")

# JS colorscale: blue → cyan → white → yellow → red
eigenspace_colorscale = [
    [0.00, "rgba(0,   0, 255, 1.0)"],
    [0.25, "rgba(0, 200, 255, 1.0)"],
    [0.50, "rgba(255,255,255, 1.0)"],
    [0.75, "rgba(255,200,  0, 1.0)"],
    [1.00, "rgba(255,  0,  0, 1.0)"],
]

# ── Prepare data ────────────────────────────────────────────────────────────
# 53-TET chords: already computed at 220 Hz in df_53tet
df_ref = df_53tet[df_53tet['in_tetrahedron']].copy()

# "Something" chords: compute D at 220 Hz (same reference as the map)
df_song = df_something[df_something['in_tetrahedron'] & df_something['dissonance'].notna()].copy()
df_song['d_220'] = [
    compute_dissonance(row['alpha'], row['beta'], row['gamma'], base_freq=220.0)
    for _, row in df_song.iterrows()
]

# ── Match each song chord to nearest 53-TET chord name ──────────────────────
ref_pos = df_ref[['alpha', 'beta', 'gamma']].values   # (N_ref, 3)
song_pos = df_song[['alpha', 'beta', 'gamma']].values  # (N_song, 3)

# Euclidean distance in (α, β, γ) space: (N_song, N_ref)
dists = np.sqrt(((song_pos[:, None, :] - ref_pos[None, :, :]) ** 2).sum(axis=2))
nearest_idx = dists.argmin(axis=1)
df_song['matched_name'] = df_ref['name'].values[nearest_idx]
df_song['match_dist'] = dists.min(axis=1)

print(f"\n53-TET chords (in tetrahedron): {len(df_ref)}")
print(f"  D range: {df_ref['dissonance'].min():.3f} – {df_ref['dissonance'].max():.3f}")
print(f"'Something' chords (in tetrahedron): {len(df_song)}")
print(f"  D at 220 Hz: {df_song['d_220'].min():.3f} – {df_song['d_220'].max():.3f}")
print(f"  Matched chord names (top 10): {df_song['matched_name'].value_counts().head(10).to_dict()}")
print(f"  Max match distance: {df_song['match_dist'].max():.6f}")

# ── Build figure ────────────────────────────────────────────────────────────
fig = go.Figure()

# Layer 1: 53-TET reference chords (circles, like JS)
fig.add_trace(go.Scatter3d(
    x=df_ref['alpha'], y=df_ref['beta'], z=df_ref['gamma'],
    mode='markers+text',
    marker=dict(
        size=5,
        color=df_ref['dissonance'],
        colorscale=eigenspace_colorscale,
        cmin=D_MAP_MIN,
        cmax=D_MAP_MAX,
        symbol='circle',
        opacity=0.9,
        line=dict(width=0.5, color='gray'),
    ),
    text=df_ref['name'],
    textposition='top center',
    textfont=dict(size=7, color='rgba(100,100,100,0.6)', family='Source Code Pro'),
    customdata=[
        f"{row['name']} (D: {row['dissonance']:.3f})"
        for _, row in df_ref.iterrows()
    ],
    hovertemplate=(
        '<b>%{customdata}</b><br>'
        'α = %{x:.4f}<br>β = %{y:.4f}<br>γ = %{z:.4f}<extra></extra>'
    ),
    name=f'53-TET Chords ({len(df_ref)})',
    showlegend=True,
))

# Layer 2: Song trajectory line
fig.add_trace(go.Scatter3d(
    x=df_song['alpha'], y=df_song['beta'], z=df_song['gamma'],
    mode='lines',
    line=dict(color='rgba(80, 80, 80, 0.3)', width=2),
    name='Song trajectory',
    showlegend=True,
    hoverinfo='skip',
))

# Layer 3: "Something" chords — same color scale
fig.add_trace(go.Scatter3d(
    x=df_song['alpha'], y=df_song['beta'], z=df_song['gamma'],
    mode='markers',
    marker=dict(
        size=6,
        color=df_song['d_220'],
        colorscale=eigenspace_colorscale,
        cmin=D_MAP_MIN,
        cmax=D_MAP_MAX,
        colorbar=dict(
            title='Dissonance (220 Hz)',
            thickness=15, len=0.6,
            tickfont=dict(size=10, family='Source Code Pro'),
            tickformat='.1f',
        ),
        symbol='diamond',
        opacity=0.95,
        line=dict(width=1, color='black'),
    ),
    customdata=[
        f"{row['matched_name']} (D: {row['d_220']:.3f})"
        for _, row in df_song.iterrows()
    ],
    text=[
        f"{row['matched_name']}<br>"
        f"α={row['alpha']:.4f} β={row['beta']:.4f} γ={row['gamma']:.4f}<br>"
        f"D={row['d_220']:.3f}"
        for _, row in df_song.iterrows()
    ],
    hoverinfo='text',
    name=f'"Something" ({len(df_song)} chords)',
    showlegend=True,
))

fig.update_layout(
    title=dict(
        text="EigenSpace — 53-TET Chords + 'Something' Trajectory<br>"
             f"<sub>Color: map p5/p80 = [{D_MAP_MIN:.2f}, {D_MAP_MAX:.2f}] · 220 Hz · 6 harmonics</sub>",
        font=dict(size=14, family='Source Code Pro'),
    ),
    scene=dict(
        xaxis=dict(title='α (2nd note ratio)', range=[1.0, 2.0],
                   backgroundcolor='white', gridcolor='lightgray',
                   tickfont=dict(family='Source Code Pro', size=9)),
        yaxis=dict(title='β (3rd note ratio)', range=[1.0, 2.0],
                   backgroundcolor='white', gridcolor='lightgray',
                   tickfont=dict(family='Source Code Pro', size=9)),
        zaxis=dict(title='γ (4th note ratio)', range=[1.0, 2.0],
                   backgroundcolor='white', gridcolor='lightgray',
                   tickfont=dict(family='Source Code Pro', size=9)),
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.0)),
        bgcolor='white',
        aspectmode='cube',
    ),
    width=1000, height=800,
    paper_bgcolor='white',
    legend=dict(
        x=0.02, y=0.98,
        font=dict(size=11, family='Source Code Pro'),
        bgcolor='rgba(255,255,255,0.8)',
    ),
)

fig.show()

print(f"\nColor scale: [{D_MAP_MIN:.2f}, {D_MAP_MAX:.2f}] from 150³ EigenSpace map")

EigenSpace color scale (from 150³ map at 220 Hz):
  cmin = 14.2065  (percentile 5)
  cmax = 19.6967  (percentile 80)
  Map absolute range: 0.6725 – 30.6237

53-TET chords (in tetrahedron): 168
  D range: 9.830 – 18.147
'Something' chords (in tetrahedron): 165
  D at 220 Hz: 9.830 – 15.985
  Matched chord names (top 10): {'MP': 36, 'vMP': 33, 'mP': 18, 'Mm7': 12, 'M+tri': 12, 'vMm7': 12, 'sus4P': 12, 'maj7': 6, 'M+m7': 6, 'ø-mtri': 6}
  Max match distance: 0.067960



Color scale: [14.21, 19.70] from 150³ EigenSpace map


In [213]:
# ============================================================================
# DISSONANCE TIMELINE — 'Something' tension evolution
# ============================================================================
# Uses df_song from cell 16: already has d_220, matched_name, in tetrahedron
# Color and Y-axis from the EigenSpace map scale (p5/p80)

fig2 = go.Figure()

# Reference band: map scale
fig2.add_hrect(
    y0=D_MAP_MIN, y1=D_MAP_MAX,
    fillcolor='rgba(180, 200, 255, 0.12)',
    line_width=0,
    annotation_text=f'Map p5–p80 [{D_MAP_MIN:.1f}, {D_MAP_MAX:.1f}]',
    annotation_position='top left',
    annotation=dict(font=dict(size=9, color='gray', family='Source Code Pro')),
)

# Song dissonance at 220 Hz — using df_song which has matched_name
fig2.add_trace(go.Scatter(
    x=df_song['beat'],
    y=df_song['d_220'],
    mode='lines+markers',
    marker=dict(
        size=5,
        color=df_song['d_220'],
        colorscale=eigenspace_colorscale,
        cmin=D_MAP_MIN,
        cmax=D_MAP_MAX,
        line=dict(width=0.5, color='black'),
        colorbar=dict(
            title='D (220 Hz)',
            thickness=12, len=0.5,
            x=1.05,
            tickfont=dict(size=9, family='Source Code Pro'),
            tickformat='.1f',
        ),
    ),
    line=dict(color='gray', width=1),
    text=[
        f"{row['matched_name']}<br>D={row['d_220']:.3f}"
        for _, row in df_song.iterrows()
    ],
    hoverinfo='text',
    name='D at 220 Hz',
))

# Smoothed trend
window = 5
if len(df_song) > window:
    smoothed = df_song['d_220'].rolling(window, center=True).mean()
    fig2.add_trace(go.Scatter(
        x=df_song['beat'], y=smoothed,
        mode='lines',
        line=dict(color='rgba(255, 0, 0, 0.4)', width=3),
        name=f'Smoothed (w={window})',
    ))

fig2.update_layout(
    title=dict(
        text="'Something' — Dissonance Timeline<br>"
             f"<sub>D at 220 Hz · map scale [{D_MAP_MIN:.1f}, {D_MAP_MAX:.1f}]</sub>",
        font=dict(size=14, family='Source Code Pro'),
    ),
    xaxis=dict(title='Beat', gridcolor='lightgray'),
    yaxis=dict(
        title='Dissonance D',
        gridcolor='lightgray',
        range=[
            min(df_song['d_220'].min(), D_MAP_MIN) - 1.0,
            max(df_song['d_220'].max(), D_MAP_MAX) + 1.0,
        ],
    ),
    width=1200, height=400,
    paper_bgcolor='white', plot_bgcolor='white',
    legend=dict(x=0.02, y=0.98, font=dict(family='Source Code Pro', size=11)),
)

fig2.show()

print(f"\n'Something' D at 220 Hz:  {df_song['d_220'].min():.3f} – {df_song['d_220'].max():.3f}  (mean {df_song['d_220'].mean():.3f})")
print(f"Map scale: [{D_MAP_MIN:.3f}, {D_MAP_MAX:.3f}]")
print(f"Chords shown: {len(df_song)} (in tetrahedron)")


'Something' D at 220 Hz:  9.830 – 15.985  (mean 12.299)
Map scale: [14.207, 19.697]
Chords shown: 165 (in tetrahedron)


In [214]:
# ============================================================================
# COMPARE "Something" — type_0 (major) vs type_1 (minor) vs type_1 (neutral)
# ============================================================================

midi_dir = "../dataset/midi_files/53_tet_mpe/"
variants = {
    "type_0 major":   f"{midi_dir}47832_Something_C_major_type_0_major.mid",
    "type_1 minor":   f"{midi_dir}47832_Something_C_major_type_1_minor.mid",
    "type_1 neutral": f"{midi_dir}47832_Something_C_major_type_1_neutral.mid",
}

dfs = {}
for label, path in variants.items():
    df_v = midi_to_eigenspace(path, ref_chords=df_ref)
    df_v = df_v[df_v['in_tetrahedron']].copy()
    dfs[label] = df_v

# ── Summary table ───────────────────────────────────────────────────────────
print(f"{'Variant':18s}  {'Chords':>6}  {'D@220 range':>18}  {'D@root range':>18}  {'Top chord names'}")
print("=" * 110)
for label, df_v in dfs.items():
    top3 = ", ".join(f"{n}×{c}" for n, c in df_v['matched_name'].value_counts().head(3).items())
    print(f"{label:18s}  {len(df_v):6d}  "
          f"{df_v['d_220'].min():7.3f} – {df_v['d_220'].max():.3f}  "
          f"{df_v['d_root'].min():7.3f} – {df_v['d_root'].max():.3f}  "
          f"{top3}")

# ── Overlaid timeline ───────────────────────────────────────────────────────
# Color = EigenSpace dissonance scale · line dash = variant type
line_dashes = {"type_0 major": "solid", "type_1 minor": "dash", "type_1 neutral": "dot"}
marker_symbols_2d = {"type_0 major": "circle", "type_1 minor": "diamond", "type_1 neutral": "square"}

fig3 = go.Figure()

# Map reference band
fig3.add_hrect(
    y0=D_MAP_MIN, y1=D_MAP_MAX,
    fillcolor='rgba(180, 200, 255, 0.10)', line_width=0,
    annotation_text='Map p5–p80', annotation_position='top left',
    annotation=dict(font=dict(size=9, color='gray', family='Source Code Pro')),
)

all_d = []
show_colorbar = True
for label, df_v in dfs.items():
    all_d.extend(df_v['d_220'].tolist())
    cb = dict(
        title='D (220 Hz)', thickness=12, len=0.5, x=1.05,
        tickfont=dict(size=9, family='Source Code Pro'), tickformat='.1f',
    ) if show_colorbar else None
    fig3.add_trace(go.Scatter(
        x=df_v['beat'], y=df_v['d_220'],
        mode='lines+markers',
        marker=dict(
            size=6,
            symbol=marker_symbols_2d[label],
            color=df_v['d_220'],
            colorscale=eigenspace_colorscale,
            cmin=D_MAP_MIN,
            cmax=D_MAP_MAX,
            line=dict(width=0.5, color='black'),
            colorbar=cb,
        ),
        line=dict(color='rgba(120,120,120,0.4)', width=1, dash=line_dashes[label]),
        text=[f"{label}<br>{row['matched_name']}<br>D={row['d_220']:.3f}" for _, row in df_v.iterrows()],
        hoverinfo='text',
        name=label,
    ))
    show_colorbar = False

all_d = np.array(all_d)
fig3.update_layout(
    title=dict(
        text="'Something' — Type Comparison (D at 220 Hz)<br>"
             f"<sub>Color: EigenSpace map [{D_MAP_MIN:.1f}, {D_MAP_MAX:.1f}] · line style = variant</sub>",
        font=dict(size=14, family='Source Code Pro'),
    ),
    xaxis=dict(title='Beat', gridcolor='lightgray'),
    yaxis=dict(
        title='Dissonance D (220 Hz)',
        gridcolor='lightgray',
        range=[min(all_d.min(), D_MAP_MIN) - 1.0, max(all_d.max(), D_MAP_MAX) + 1.0],
    ),
    width=1200, height=450,
    paper_bgcolor='white', plot_bgcolor='white',
    legend=dict(x=0.02, y=0.98, font=dict(family='Source Code Pro', size=11)),
)
fig3.show()

# ── EigenSpace 3D comparison ───────────────────────────────────────────────
fig4 = go.Figure()

# 53-TET reference — colored by EigenSpace dissonance (faint)
fig4.add_trace(go.Scatter3d(
    x=df_ref['alpha'], y=df_ref['beta'], z=df_ref['gamma'],
    mode='markers',
    marker=dict(
        size=3,
        color=df_ref['dissonance'],
        colorscale=eigenspace_colorscale,
        cmin=D_MAP_MIN, cmax=D_MAP_MAX,
        opacity=0.25,
    ),
    name='53-TET', showlegend=True, hoverinfo='skip',
))

symbols_3d = {"type_0 major": "circle", "type_1 minor": "diamond", "type_1 neutral": "square"}
show_cbar = True
for label, df_v in dfs.items():
    cb3 = dict(
        title='D (220 Hz)', thickness=12, len=0.5,
        tickfont=dict(size=9, family='Source Code Pro'), tickformat='.1f',
    ) if show_cbar else None
    fig4.add_trace(go.Scatter3d(
        x=df_v['alpha'], y=df_v['beta'], z=df_v['gamma'],
        mode='markers',
        marker=dict(
            size=6,
            color=df_v['d_220'],
            colorscale=eigenspace_colorscale,
            cmin=D_MAP_MIN, cmax=D_MAP_MAX,
            opacity=0.9,
            symbol=symbols_3d[label],
            line=dict(width=0.5, color='black'),
            colorbar=cb3,
        ),
        text=[f"{label}<br>{row['matched_name']}<br>D={row['d_220']:.3f}" for _, row in df_v.iterrows()],
        hoverinfo='text',
        name=label,
    ))
    show_cbar = False

fig4.update_layout(
    title=dict(
        text="EigenSpace — 'Something' Voicing Types<br>"
             f"<sub>Color: map [{D_MAP_MIN:.1f}, {D_MAP_MAX:.1f}] · symbol = variant</sub>",
        font=dict(size=14, family='Source Code Pro'),
    ),
    scene=dict(
        xaxis=dict(title='α', range=[1.0, 2.0], backgroundcolor='white', gridcolor='lightgray'),
        yaxis=dict(title='β', range=[1.0, 2.0], backgroundcolor='white', gridcolor='lightgray'),
        zaxis=dict(title='γ', range=[1.0, 2.0], backgroundcolor='white', gridcolor='lightgray'),
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.0)),
        bgcolor='white', aspectmode='cube',
    ),
    width=900, height=700,
    paper_bgcolor='white',
    legend=dict(x=0.02, y=0.98, font=dict(size=11, family='Source Code Pro')),
)
fig4.show()

Variant             Chords         D@220 range        D@root range  Top chord names
type_0 major           165    9.830 – 15.985   13.961 – 41.928  MP×36, vMP×33, mP×18
type_1 minor           165    9.830 – 15.528   12.683 – 38.829  ^mP×36, ^MP×18, nP×18
type_1 neutral         165    9.830 – 15.443   12.487 – 42.699  nP×24, MP×21, vMP×18


In [215]:
# ============================================================================
# EIGENSPACE MAPPING INTEGRITY CHECK
# ============================================================================
# Verify: ALL song chords map to exact 53-TET vocabulary positions.
# Every (α, β, γ) must correspond to a point in the vocabulary grid.

# ─── Build vocabulary lookup ─────────────────────────────────────────────
vocab_positions = set()
for name, a, b, g in chords_53tet:
    vocab_positions.add((round(a, 6), round(b, 6), round(g, 6)))

# ─── Check every song chord ─────────────────────────────────────────────
on_grid = 0
off_grid_examples = []

for i, ch in enumerate(raw_chords):
    cl = classify_chord_intervals(ch['intervals'])
    pos = (round(cl['alpha'], 6), round(cl['beta'], 6), round(cl['gamma'], 6))
    
    if pos in vocab_positions:
        on_grid += 1
    else:
        off_grid_examples.append({
            'idx': i, 'beat': ch['beat'],
            'intervals': ch['intervals'],
            'alpha': cl['alpha'], 'beta': cl['beta'], 'gamma': cl['gamma'],
            'third': cl['third'], 'fifth': cl['fifth'], 'seventh': cl['seventh'],
            'type': cl['chord_type'],
        })

print(f"EIGENSPACE INTEGRITY CHECK — '{song_name}'")
print(f"=" * 100)
print(f"  Total chords:         {len(raw_chords)}")
print(f"  On vocabulary grid:   {on_grid}")
print(f"  Off vocabulary grid:  {len(off_grid_examples)}")
print(f"  Vocabulary positions: {len(vocab_positions)}")

if off_grid_examples:
    print(f"\n  OFF-GRID CHORDS (not matching any vocabulary position):")
    for ex in off_grid_examples[:10]:
        print(f"    beat {ex['beat']:5.1f}: {ex['type']:16s}  "
              f"α={ex['alpha']:.6f} β={ex['beta']:.6f} γ={ex['gamma']:.6f}  "
              f"steps=[{ex['third']},{ex['fifth']},{ex['seventh']}]  iv={ex['intervals']}")
    
    # Investigate: find nearest vocabulary chord for each
    print(f"\n  NEAREST MATCHES:")
    ref_arr = np.array([(a, b, g) for a, b, g in vocab_positions])
    for ex in off_grid_examples[:10]:
        p = np.array([ex['alpha'], ex['beta'], ex['gamma']])
        dists = np.sqrt(((ref_arr - p) ** 2).sum(axis=1))
        min_dist = dists.min()
        min_idx = dists.argmin()
        nearest = ref_arr[min_idx]
        print(f"    → nearest: α={nearest[0]:.6f} β={nearest[1]:.6f} γ={nearest[2]:.6f}  (dist={min_dist:.6f})")
else:
    print(f"\n  ✓ EVERY chord maps to an exact 53-TET vocabulary grid position.")
    print(f"    This means the harmonic motion trajectory is fully quantized")
    print(f"    to the {len(vocab_positions)} points in EigenSpace.")

EIGENSPACE INTEGRITY CHECK — 'Something'
  Total chords:         165
  On vocabulary grid:   147
  Off vocabulary grid:  18
  Vocabulary positions: 168

  OFF-GRID CHORDS (not matching any vocabulary position):
    beat  38.0: sus4-+-–          α=1.333386 β=1.580496 γ=1.580496  steps=[22,35,None]  iv=[0, 4, 22, 35]
    beat  50.0: sus4-+-m7         α=1.333386 β=1.580496 γ=1.777918  steps=[22,35,44]  iv=[0, 9, 22, 35, 44]
    beat  92.0: SM-+-–            α=1.298961 β=1.580496 γ=1.580496  steps=[20,35,None]  iv=[0, 4, 21, 35]
    beat 124.0: SM-+-–            α=1.298961 β=1.580496 γ=1.580496  steps=[20,35,None]  iv=[0, 4, 21, 35]
    beat 182.0: sus4-+-–          α=1.333386 β=1.580496 γ=1.580496  steps=[22,35,None]  iv=[0, 4, 22, 35]
    beat 194.0: sus4-+-m7         α=1.333386 β=1.580496 γ=1.777918  steps=[22,35,44]  iv=[0, 9, 22, 35, 44]
    beat 286.0: sus4-+-–          α=1.333386 β=1.580496 γ=1.580496  steps=[22,35,None]  iv=[0, 4, 22, 35]
    beat 298.0: sus4-+-m7         α=1.33338

# Pipeline Validation: `midi_to_eigenspace()` — Any Song → 4D Coordinates

The complete method that will be integrated into the transformer's positional embedding:

```
MPE MIDI → parse chords → classify intervals → (α, β, γ) ratios
                                                       ↓
                                              compute D at root Hz (Plomp-Levelt)
                                                       ↓
                                        match to nearest 53-TET chord name
                                                       ↓
                                              (α, β, γ, D) per chord
                                                       ↓
                                           per-token embedding in GPT-2
```

**Key difference from the current `eigenspace.py`**: Dissonance D is computed with the **actual root frequency** of each chord (not a fixed 220 Hz map lookup). Low-register chords have wider critical bands → more dissonance. This is the physics.

In [191]:
# ============================================================================
# UNIFIED PIPELINE: Any MPE MIDI → EigenSpace 4D Coordinates
# ============================================================================
#
# This is the COMPLETE pipeline for mapping any song to its EigenSpace
# trajectory. It is the foundation for the transformer's harmonic embedding.
#
# Pipeline:
#   MIDI file → parse notes → fold to one octave → extract tetrad →
#   snap to 53-TET vocabulary → compute (α, β, γ) ratios →
#   compute D at root Hz → match to vocabulary name → output DataFrame
#
# The output DataFrame represents the HARMONIC MOTION of the song:
# a sequence of points in the EigenSpace tetrahedron, each with
# its own dissonance computed at its actual register.

def midi_to_eigenspace(midi_path: str,
                       ref_chords: pd.DataFrame = None,
                       num_harmonics: int = 6,
                       ) -> pd.DataFrame:
    """
    Map any 53-TET MPE MIDI file to EigenSpace 4D coordinates.
    
    The full pipeline:
        1. Parse MIDI → one-octave pitch classes per chord (voicing stripped)
        2. Extract tetrad → (α, β, γ) frequency ratios from canonical grid
        3. Compute Plomp-Levelt dissonance D at actual root Hz
        4. Also compute D at 220 Hz for cross-song comparison
        5. Match to nearest 53-TET chord vocabulary name
    
    Args:
        midi_path:      Path to 53-TET MPE MIDI file
        ref_chords:     53-TET chord reference DataFrame (for name matching)
        num_harmonics:  Harmonics per voice for dissonance (default 6)
    
    Returns:
        DataFrame with:
            beat, root_hz, root_step, intervals, chord_type,
            alpha, beta, gamma (ratios, all in 1.0–2.0),
            d_root (D at actual root frequency — the true 4th dimension),
            d_220  (D at 220 Hz — normalized for comparison),
            matched_name (nearest 53-TET vocabulary chord),
            match_dist, is_triad, in_tetrahedron, extras
    """
    # ── 1. Parse MIDI → one-octave pitch classes ────────────────────────────
    raw = parse_mpe_midi_chords(midi_path)
    
    # ── 2–3. Extract tetrad + compute D ─────────────────────────────────────
    rows = []
    for ch in raw:
        cl = classify_chord_intervals(ch['intervals'])
        in_tet = cl['alpha'] <= cl['beta'] <= cl['gamma']
        
        d_root = compute_dissonance(
            cl['alpha'], cl['beta'], cl['gamma'],
            base_freq=ch['root_hz'], num_harmonics=num_harmonics
        )
        d_220 = compute_dissonance(
            cl['alpha'], cl['beta'], cl['gamma'],
            base_freq=220.0, num_harmonics=num_harmonics
        )
        
        rows.append({
            'beat':       ch['beat'],
            'root_hz':    ch['root_hz'],
            'root_step':  ch['root_step'],
            'intervals':  str(ch['intervals']),
            'chord_type': cl['chord_type'],
            'third':      cl['third'],
            'fifth':      cl['fifth'],
            'seventh':    cl['seventh'],
            'alpha':      cl['alpha'],
            'beta':       cl['beta'],
            'gamma':      cl['gamma'],
            'd_root':     d_root,
            'd_220':      d_220,
            'is_triad':   cl['is_triad'],
            'in_tetrahedron': in_tet,
            'extras':     str(cl['extras']) if cl['extras'] else '',
        })
    
    df = pd.DataFrame(rows)
    
    # ── 4. Match to nearest 53-TET chord vocabulary name ────────────────────
    if ref_chords is None:
        ref_chords = map_chords_to_eigenspace(chords_53tet, alpha_range,
                                               beta_range, gamma_range,
                                               dissonance_3d)
        ref_chords = ref_chords[ref_chords['in_tetrahedron']].copy()
    
    ref_pos = ref_chords[['alpha', 'beta', 'gamma']].values
    in_tet = df[df['in_tetrahedron']].copy()
    
    if len(in_tet) > 0:
        song_pos = in_tet[['alpha', 'beta', 'gamma']].values
        dists = np.sqrt(((song_pos[:, None, :] - ref_pos[None, :, :]) ** 2).sum(axis=2))
        nearest_idx = dists.argmin(axis=1)
        
        df.loc[in_tet.index, 'matched_name'] = ref_chords['name'].values[nearest_idx]
        df.loc[in_tet.index, 'match_dist'] = dists.min(axis=1)
    
    df['matched_name'] = df['matched_name'].fillna('?')
    df['match_dist'] = df['match_dist'].fillna(np.nan)
    
    return df


# ── Test on "Something" ────────────────────────────────────────────────────
test_path = "../dataset/midi_files/53_tet_mpe/47832_Something_C_major_type_0_major.mid"
df_test = midi_to_eigenspace(test_path, ref_chords=df_ref)

print(f"midi_to_eigenspace() — '{song_name}'")
print(f"=" * 100)
print(f"  Chords:     {len(df_test)}")
print(f"  In tetrah:  {df_test['in_tetrahedron'].sum()}")
print(f"  Triads:     {df_test['is_triad'].sum()}")
print(f"  Root Hz:    {df_test['root_hz'].min():.1f} – {df_test['root_hz'].max():.1f}")
print(f"  D@root:     {df_test['d_root'].min():.3f} – {df_test['d_root'].max():.3f}")
print(f"  D@220:      {df_test['d_220'].min():.3f} – {df_test['d_220'].max():.3f}")
print(f"  Matched:    {df_test[df_test['matched_name'] != '?']['matched_name'].nunique()} unique chord names")
print()
print(f"  Top chord names:")
for name, cnt in df_test['matched_name'].value_counts().head(10).items():
    print(f"    {name:12s}  ×{cnt}")

print(f"\n  First 10 chords:")
print(f"  {'#':>3}  {'Beat':>5}  {'Root Hz':>7}  {'Name':12}  {'α':>7}  {'β':>7}  {'γ':>7}  {'D@root':>7}  {'D@220':>7}")
print(f"  " + "-" * 80)
for i, (_, row) in enumerate(df_test.head(10).iterrows()):
    print(f"  {i+1:3d}  {row['beat']:5.1f}  {row['root_hz']:7.1f}  {row['matched_name']:12s}  "
          f"{row['alpha']:7.4f}  {row['beta']:7.4f}  {row['gamma']:7.4f}  "
          f"{row['d_root']:7.3f}  {row['d_220']:7.3f}")

midi_to_eigenspace() — 'Something' (Beatles)
  Chords:     165
  In tetrah:  165
  Triads:     117
  Root Hz:    65.4 – 174.4
  D@root:     13.989 – 41.928
  D@220:      8.821 – 15.661
  Matched:    12 unique chord names

  Top chord names:
    MSM7          ×36
    vMSM7         ×33
    mSM7          ×18
    Mm7           ×12
    M+S7          ×12
    vMm7          ×12
    SMSM7         ×12
    maj7          ×6
    M+m7          ×6
    øS7           ×6

  First 10 chords:
    #   Beat  Root Hz  Name                α        β        γ   D@root    D@220
  --------------------------------------------------------------------------------
    1    0.0    174.4  MSM7           1.2654   1.4999   2.0000   13.989   11.592
    2    4.0    157.1  vMSM7          1.2490   1.4999   2.0000   14.245   10.429
    3    6.0    147.2  mSM7           1.1853   1.4999   2.0000   15.728   10.928
    4    8.0    130.8  MSM7           1.2654   1.4999   2.0000   18.120   11.592
    5   16.0    130.8  maj7       

In [192]:
# ============================================================================
# CROSS-VALIDATION: Run pipeline on 3 different songs from the dataset
# ============================================================================

test_songs = [
    ("A Night In Tunisia", "../dataset/midi_files/53_tet_mpe/10716_A Night In Tunisia_C_minor_type_0_major.mid"),
    ("My Girl",            "../dataset/midi_files/53_tet_mpe/10920_My Girl_C_major_type_0_major.mid"),
    ("So Sweet To Trust",  "../dataset/midi_files/53_tet_mpe/23897_So Sweet To Trust In Jesus_F_major_type_0_major.mid"),
]

all_results = {}
print(f"{'Song':25s}  {'Chords':>6}  {'InTet':>5}  {'RootHz':>14}  {'D@root':>16}  {'D@220':>16}  {'Names':>5}")
print("=" * 110)

for song_name, song_path in test_songs:
    try:
        df = midi_to_eigenspace(song_path, ref_chords=df_ref)
        all_results[song_name] = df
        in_tet = df['in_tetrahedron'].sum()
        print(
            f"{song_name:25s}  {len(df):6d}  {in_tet:5d}  "
            f"{df['root_hz'].min():6.1f}–{df['root_hz'].max():6.1f}  "
            f"{df['d_root'].min():7.3f}–{df['d_root'].max():7.3f}  "
            f"{df['d_220'].min():7.3f}–{df['d_220'].max():7.3f}  "
            f"{df[df['matched_name']!='?']['matched_name'].nunique():5d}"
        )
    except Exception as e:
        print(f"{song_name:25s}  ERROR: {e}")

# ── Detailed look: A Night In Tunisia ───────────────────────────────────────
if "A Night In Tunisia" in all_results:
    df_tunisia = all_results["A Night In Tunisia"]
    print(f"\n\n'A Night In Tunisia' — detailed")
    print(f"-" * 90)
    print(f"  {'#':>3}  {'Beat':>5}  {'RootHz':>7}  {'Name':12}  {'α':>7}  {'β':>7}  {'γ':>7}  {'D@root':>7}  {'D@220':>7}  {'Triad':>5}")
    print(f"  " + "-" * 85)
    for i, (_, row) in enumerate(df_tunisia[df_tunisia['in_tetrahedron']].head(20).iterrows()):
        t = "triad" if row['is_triad'] else ""
        print(f"  {i+1:3d}  {row['beat']:5.1f}  {row['root_hz']:7.1f}  {row['matched_name']:12s}  "
              f"{row['alpha']:7.4f}  {row['beta']:7.4f}  {row['gamma']:7.4f}  "
              f"{row['d_root']:7.3f}  {row['d_220']:7.3f}  {t}")

# ── Verify D@root vs D@220 divergence (the whole point) ────────────────────
print("\n\n" + "=" * 90)
print("FREQUENCY DEPENDENCE VERIFICATION")
print("=" * 90)
print("D@root vs D@220 — same (α,β,γ); different root pitch → different D")
print()
for song_name, df in all_results.items():
    tet = df[df['in_tetrahedron']].copy()
    if len(tet) == 0:
        continue
    ratio = tet['d_root'] / tet['d_220']
    print(f"  {song_name:25s}  D_root/D_220 = {ratio.min():.3f} – {ratio.max():.3f}  "
          f"(mean {ratio.mean():.3f})")
    # Correlation between root Hz and D_root/D_220
    corr = np.corrcoef(tet['root_hz'], ratio)[0, 1]
    print(f"  {'':25s}  corr(root_hz, ratio) = {corr:.3f}  "
          f"({'lower pitch → more dissonance ✓' if corr < -0.5 else 'check'})")

print("\n✓ Lower root frequencies → higher D_root relative to D_220")
print("  This is the Plomp-Levelt critical bandwidth effect.")

Song                       Chords  InTet          RootHz            D@root             D@220  Names
A Night In Tunisia            129    129    65.4– 147.2   18.120– 41.928   11.592– 17.705     11
My Girl                       162    162    65.4– 196.2    9.985– 36.256    8.821– 14.911      6
So Sweet To Trust              93     93    73.6– 155.1   13.776– 35.539    4.218– 14.911      9


'A Night In Tunisia' — detailed
------------------------------------------------------------------------------------------
    #   Beat   RootHz  Name                α        β        γ   D@root    D@220  Triad
  -------------------------------------------------------------------------------------
    1    0.0    137.8  MN7            1.2654   1.4999   1.8491   21.198   15.207  
    2    8.0    130.8  Mm7            1.2654   1.4999   1.7779   22.673   14.911  
    3   16.0    137.8  MN7            1.2654   1.4999   1.8491   21.198   15.207  
    4   24.0    130.8  Mm7            1.2654   1.4999   1.7

In [193]:
# ============================================================================
# OUTPUT SHAPE FOR TRANSFORMER: per-chord → per-token 4D vectors
# ============================================================================
# The transformer needs a (seq_len, 4) array — one (α, β, γ, D) per token.
# midi_to_eigenspace() gives us per-CHORD coordinates.
# The tokenizer (05_midi_mpe_tokenization.py) produces CHORD_START...CHORD_END blocks.
# Every token inside a chord block inherits that chord's 4D vector.
#
# This is the BRIDGE between the notebook pipeline and eigenspace.py:
#   1. midi_to_eigenspace()  → per-chord DataFrame  (what we validated here)
#   2. EigenSpaceComputer    → per-token (seq_len, 4) array  (for the model)
#
# What needs to change in eigenspace.py:
#   - Replace DissonanceMap.lookup(α,β,γ) with compute_dissonance(α,β,γ, root_hz)
#   - Extract root_hz from the pitch tokens (min P_* step → step53_to_hz)
#   - Keep the per-token broadcast logic (CHORD_START → CHORD_END)

# ── Demonstrate the output shape ────────────────────────────────────────────
# For "A Night In Tunisia": what the model will see per chord position
df_demo = all_results["A Night In Tunisia"]
in_tet = df_demo[df_demo['in_tetrahedron']].copy()

print("EigenSpace 4D vectors — what the transformer will receive per chord position")
print("=" * 80)
print(f"  Shape per song: ({len(in_tet)}, 4)  →  broadcast to (seq_len, 4) per token\n")

# Show the 4D vector for each chord
vectors = in_tet[['alpha', 'beta', 'gamma', 'd_root']].values
print(f"  {'Chord':12s}  {'α':>8s}  {'β':>8s}  {'γ':>8s}  {'D':>8s}")
print(f"  " + "-" * 50)
for i, (_, row) in enumerate(in_tet.head(10).iterrows()):
    print(f"  {row['matched_name']:12s}  {row['alpha']:8.4f}  {row['beta']:8.4f}  {row['gamma']:8.4f}  {row['d_root']:8.3f}")

print(f"\n  ... {len(in_tet)} vectors total")
print(f"\n  D range:  {vectors[:, 3].min():.3f} – {vectors[:, 3].max():.3f}")
print(f"  D mean:   {vectors[:, 3].mean():.3f}")
print(f"  D std:    {vectors[:, 3].std():.3f}")

# ── Statistics across all validated songs ────────────────────────────────────
print("\n\nACROSS ALL VALIDATED SONGS")
print("=" * 80)
all_d_root = []
all_d_220 = []
for name, df in {**all_results, "Something": df_test}.items():
    tet = df[df['in_tetrahedron']]
    all_d_root.extend(tet['d_root'].tolist())
    all_d_220.extend(tet['d_220'].tolist())

all_d_root = np.array(all_d_root)
all_d_220 = np.array(all_d_220)

print(f"  Total chords pooled:  {len(all_d_root)}")
print(f"  D@root: mean={all_d_root.mean():.3f}, std={all_d_root.std():.3f}, range=[{all_d_root.min():.3f}, {all_d_root.max():.3f}]")
print(f"  D@220:  mean={all_d_220.mean():.3f}, std={all_d_220.std():.3f}, range=[{all_d_220.min():.3f}, {all_d_220.max():.3f}]")

print(f"\n  For z-normalization in the embedding:")
print(f"    D_mean = {all_d_root.mean():.4f}")
print(f"    D_std  = {all_d_root.std():.4f}")
print(f"    Normalized range: [{(all_d_root.min() - all_d_root.mean()) / all_d_root.std():.2f}, "
      f"{(all_d_root.max() - all_d_root.mean()) / all_d_root.std():.2f}]")

EigenSpace 4D vectors — what the transformer will receive per chord position
  Shape per song: (129, 4)  →  broadcast to (seq_len, 4) per token

  Chord                α         β         γ         D
  --------------------------------------------------
  MN7             1.2654    1.4999    1.8491    21.198
  Mm7             1.2654    1.4999    1.7779    22.673
  MN7             1.2654    1.4999    1.8491    21.198
  Mm7             1.2654    1.4999    1.7779    22.673
  MN7             1.2654    1.4999    1.8491    21.198
  Mm7             1.2654    1.4999    1.7779    22.673
  m7              1.1853    1.4999    1.7779    18.838
  SMm7            1.2990    1.4999    1.7779    29.468
  MSM7            1.2654    1.4999    2.0000    18.120
  ø7              1.2654    1.4050    1.7779    32.219

  ... 129 vectors total

  D range:  18.120 – 41.928
  D mean:   30.855
  D std:    7.179


ACROSS ALL VALIDATED SONGS
  Total chords pooled:  549
  D@root: mean=26.280, std=7.724, range=[9.985, 4